In [1]:
# CONFIG
KAGGLE = True
DEBUG  = False
SAVE_SW = True
YEAR_FILTER = True
LEAKAGE_FILTER = True
RETRAIN = False
RESAMPLE = True
ADV = True
VALID = False
SUPPLEMENT = True
basesize = 2236494
SUBTEST = False
savepath = '../input/g-r-finaldataset'

if KAGGLE:
  save_directry = './'
else:
  save_directry = '/content/drive/MyDrive/Machine_Learning/data/kaggle/2021_gresearch_crypto/04_model/26_23codehosei'

title_num = 26

In [2]:
# 3601のところを高速化
# 2019で実施
# RETRAIN = True
# SUBTESTをFalseに変更

In [3]:
import multiprocessing
multiprocessing.cpu_count()

4

In [4]:
if KAGGLE:
  import gresearch_crypto
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from datetime import datetime

from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from scipy.stats import pearsonr, spearmanr
import gc
import pickle
import matplotlib.pyplot as plt
import glob
import psutil

from sklearn.pipeline import Pipeline

def timestamp_to_date(timestamp):
    return(datetime.fromtimestamp(timestamp))


if KAGGLE:
  import os
  for dirname, _, filenames in os.walk('/kaggle/input'):
      for filename in filenames:
          print(os.path.join(dirname, filename))




#env = gresearch_crypto.make_env()

#iter_test = env.iter_test()

#(test_df, sample_prediction_df) = next(iter_test)

/kaggle/input/g-research-46-maketrainfull-set2020start/2020_20210920.parquet
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM_macd.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/__results__.html
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM_17.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM_lag.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM_market.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/__notebook__.ipynb
/kaggle/input/g-research-46-maketrainfull-set2020start/__output__.json
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM_richbeta.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM_vols.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/26_dict_RM_base.pkl
/kaggle/input/g-research-46-maketrainfull-set2020start/custom.css
/kaggle/input/28-lgbm/28_lgbm_models/lgbm_

In [5]:
if KAGGLE:
  pass
else:
  from google.colab import drive
  drive.mount('/content/drive')

In [6]:
# peak memory
import os
import re

def peak_memory():
    pid = os.getpid()
    with open(f'/proc/{pid}/status') as f:
        for line in f:
            if not line.startswith('VmHWM:'):
                continue
            return int(re.search('[0-9]+', line)[0])
    raise ValueError('Not Found')

def f():
    a = [0] * 20000000

<a id='Folds'></a>
# Creating Training Folds

For the design of the folds, see discussion here: https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/288555

In [7]:
if RETRAIN:
    #nrows = 5000000 if DEBUG else None
    #nrows = 12000000 if DEBUG else None
    #nrows = 100000 if DEBUG else None
    nrows = 300000 if DEBUG else None

    dtype={'Asset_ID': 'int8', 'Count': 'int32', 'row_id': 'int32', 'Count': 'int32',
           'Open': 'float32', 'High': 'float32', 'Low': 'float32', 'Close': 'float32',
           'Volume': 'float32', 'VWAP': 'float32'}

    if KAGGLE:
      #train_df = pd.read_csv('../input/g-research-crypto-forecasting/train.csv', low_memory=False, dtype=dtype, nrows=nrows)
      asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
      train_df = pd.read_parquet('../input/gresearch-recalc-trandata/train_recalc.parquet')
      if DEBUG:
        train_df = train_df.iloc[:nrows, :].reset_index(drop=True) 

    else:
      #train_df = pd.read_csv('/content/drive/MyDrive/Machine_Learning/data/kaggle/2021_gresearch_crypto/01_input/train.csv', low_memory=False, dtype=dtype, nrows=nrows)
      train_df = pd.read_parquet('/content/drive/MyDrive/Machine_Learning/data/kaggle/2021_gresearch_crypto/01_input/train_recalc_2201.parquet')
      if DEBUG:
        train_df = train_df.iloc[:nrows, :].reset_index(drop=True)
    #filter to avoid time leakage with the data 
    filter_leakage = pd.to_datetime(train_df['timestamp'], unit='s') < '2021-06-13 00:00:00'
    filter_memory = pd.to_datetime(train_df['timestamp'], unit='s') >= '2019-01-01 00:00:00'
    
    if LEAKAGE_FILTER:
        train_df = train_df[filter_leakage]
    
    if YEAR_FILTER:
        train_df = train_df[filter_memory]
        train_df.reset_index(drop=True, inplace=True)

    print(train_df.shape)
    display(train_df)

In [8]:
if SUPPLEMENT:
    dtype={'Asset_ID': 'int8', 'Count': 'int32', 'row_id': 'int32', 'Count': 'int32',
           'Open': 'float32', 'High': 'float32', 'Low': 'float32', 'Close': 'float32',
           'Volume': 'float32', 'VWAP': 'float32'}
    df_read  = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv', dtype=dtype)
    
    if SUBTEST:
        filter_leakage = pd.to_datetime(df_read['timestamp'], unit='s') < '2021-12-31 00:00:00'
        filter_memory = pd.to_datetime(df_read['timestamp'], unit='s') >= '2021-09-01 00:00:00'
        df_read2 = df_read[filter_memory]
        df_read2 = df_read2[filter_leakage]
        df_read2['timestamp'] = df_read2['timestamp'] + (3600 * 24 * 30*5)
        #print(pd.to_datetime(df_read2.head(1)['timestamp'].values[0], unit='s'))
        df_read = pd.concat([df_read, df_read2], ignore_index=True, copy=False)

    if DEBUG:
        nrows = 300000 if DEBUG else None
        df_read  = df_read.iloc[:nrows, :]
    
    print(pd.to_datetime(df_read.head(1)['timestamp'].values[0], unit='s'))
    print(pd.to_datetime(df_read.tail(1)['timestamp'].values[0], unit='s'))
    print(df_read.shape)
    print('='*100)
    datasize = len(df_read)
    diff_datasize = datasize - basesize
    display(df_read.head())
    print(diff_datasize)

2021-09-21 00:01:00
2022-01-24 00:00:00
(2518278, 10)


,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1632182460,3,561,2.079028,2.080605,2.072000,2.076458,2.804627e+05,2.075869,0.003085
1,1632182460,2,169,541.005981,541.200012,539.700012,540.721985,1.889432e+02,540.716919,-0.000607
2,1632182460,0,400,363.737488,363.899994,363.000000,363.653992,8.792685e+02,363.499542,-0.019375
3,1632182460,1,1933,42986.144531,43001.000000,42898.000000,42947.066406,9.183862e+01,42942.976562,-0.000374
4,1632182460,4,348,0.208326,0.208400,0.207800,0.208200,1.051337e+06,0.208077,-0.001233


281784


In [9]:
if KAGGLE: 
  asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
else:
  asset_details = pd.read_csv('/content/drive/MyDrive/Machine_Learning/data/kaggle/2021_gresearch_crypto/01_input/asset_details.csv')

#create dictionnary of weights
dict_weights = {}
for i in range(asset_details.shape[0]):
    dict_weights[asset_details.iloc[i,0]] = asset_details.iloc[i,1]
    
weigths = np.array([dict_weights[i] for i in range(14)])

In [10]:
if RETRAIN:
    # Generate the class/group data

    import os
    time_ids = train_df.timestamp.unique()

    n_fold = 5
    splits = 0.6
    ntimes = len(time_ids)

    embargo_train_test = 100 if DEBUG else 60*24*30
    embargo_fold = 100 if DEBUG else 60*24*30

    time_per_fold = (ntimes - 5*embargo_train_test - 5*embargo_fold)/5
    train_len = splits*time_per_fold 
    test_len = (1-splits)*time_per_fold

    fold_start = [np.int(i*(len(time_ids)+1)/5) for i in range(6)]

    for i in range(n_fold):
        time_folds = time_ids[fold_start[i]:fold_start[i+1]-1]
        df_fold = train_df[train_df.timestamp.isin(time_folds)]
        df_fold.to_parquet('df_fold_'+str(i)+'.parquet')

    del train_df
    gc.collect()

    dict_fold = {}

    for fold in range(n_fold):
        print('fold:'+str(fold))

        df_fold = pd.read_parquet('df_fold_'+str(fold)+'.parquet')
        time_ids = df_fold.timestamp.unique()

        test_train_len = len(time_ids) - embargo_train_test - embargo_fold

        train_start = embargo_fold + 1
        train_end = embargo_fold + np.int(test_train_len*0.6) + 1
        test_start = embargo_fold + np.int(test_train_len*0.6) + embargo_train_test + 1
        test_end = len(df_fold.timestamp.unique())

        dict_fold['train_fold_'+str(fold)] = time_ids[train_start:train_end]
        dict_fold['test_fold_'+str(fold)] = time_ids[test_start:test_end]

    del df_fold
    gc.collect()

'''
if SUPPLEMENT:
    # Generate the class/group data

    import os
    time_ids = supple.timestamp.unique()

    n_fold = 1
    splits = 0.6
    ntimes = len(time_ids)

    embargo_train_test = 100 if DEBUG else 60*24*30
    embargo_fold = 100 if DEBUG else 60*24*30

    time_per_fold = (ntimes - 5*embargo_train_test - 5*embargo_fold)/5
    train_len = splits*time_per_fold 
    test_len = (1-splits)*time_per_fold

    fold_start = [np.int(i*(len(time_ids)+1)/5) for i in range(6)]

    for i in range(n_fold):
        time_folds = time_ids[fold_start[i]:fold_start[i+1]-1]
        df_fold = supple[supple.timestamp.isin(time_folds)]
        df_fold.to_parquet('df_fold_'+str(i)+'.parquet')

    del supple
    gc.collect()

    dict_fold = {}

    for fold in range(n_fold):
        print('fold:'+str(fold))

        df_fold = pd.read_parquet('df_fold_'+str(fold)+'.parquet')
        time_ids = df_fold.timestamp.unique()

        test_train_len = len(time_ids) - embargo_train_test - embargo_fold

        train_start = embargo_fold + 1
        train_end = embargo_fold + np.int(test_train_len*0.6) + 1
        test_start = embargo_fold + np.int(test_train_len*0.6) + embargo_train_test + 1
        test_end = len(df_fold.timestamp.unique())

        dict_fold['train_fold_'+str(fold)] = time_ids[train_start:train_end]
        dict_fold['test_fold_'+str(fold)] = time_ids[test_start:test_end]

    del df_fold
    gc.collect()
'''

"\nif SUPPLEMENT:\n    # Generate the class/group data\n\n    import os\n    time_ids = supple.timestamp.unique()\n\n    n_fold = 1\n    splits = 0.6\n    ntimes = len(time_ids)\n\n    embargo_train_test = 100 if DEBUG else 60*24*30\n    embargo_fold = 100 if DEBUG else 60*24*30\n\n    time_per_fold = (ntimes - 5*embargo_train_test - 5*embargo_fold)/5\n    train_len = splits*time_per_fold \n    test_len = (1-splits)*time_per_fold\n\n    fold_start = [np.int(i*(len(time_ids)+1)/5) for i in range(6)]\n\n    for i in range(n_fold):\n        time_folds = time_ids[fold_start[i]:fold_start[i+1]-1]\n        df_fold = supple[supple.timestamp.isin(time_folds)]\n        df_fold.to_parquet('df_fold_'+str(i)+'.parquet')\n\n    del supple\n    gc.collect()\n\n    dict_fold = {}\n\n    for fold in range(n_fold):\n        print('fold:'+str(fold))\n\n        df_fold = pd.read_parquet('df_fold_'+str(fold)+'.parquet')\n        time_ids = df_fold.timestamp.unique()\n\n        test_train_len = len(time_id

In [11]:
# config
MA_lags = [15, 240]#,60,240,1440]#,7*24*60,30*24*60]
beta_lags = [60]#,7*24*60]#,30*24*60]
print(len(MA_lags), len(beta_lags))

2 1


In [12]:
import collections
from collections import deque

class RunningMean:
    def __init__(self, WIN_SIZE=20, n_size = 1):
        self.n = 0
        self.mean = np.zeros(n_size)
        self.cum_max = 0
        self.cum_sum = 0
        self.past_value = 0
        self.WIN_SIZE = WIN_SIZE
        self.windows = collections.deque(maxlen=WIN_SIZE+1)
        
    def clear(self):
        self.n = 0
        self.windows.clear()

    def push(self, x):
        #currently fillna with past value, might want to change that
        x = fillna_npwhere(x, self.past_value)
        self.past_value = x
        
        self.windows.append(x)
        self.cum_sum += x
        
        if self.n < self.WIN_SIZE:
            self.n += 1
            self.mean = self.cum_sum / float(self.n)
            self.max = np.max(self.windows)
            
        else:
            self.cum_sum -= self.windows.popleft()# 一番古いの削除
            self.mean = self.cum_sum / float(self.WIN_SIZE)
            #self.max = np.max(self.cum_sum)

    def get_mean(self):
        #print(windows.shape)
        return self.mean if self.n else np.zeros(n_size)
      
    def get_max(self):
        kara = []

        a = len(self.windows[0])
        #print(a)
        for i in range(len(self.windows)):
 
          kara.append(np.hstack(self.windows[i]))
        #print(np.vstack(kara).shape)
        return np.max(np.vstack(kara), axis=0).reshape(a, -1) if self.n else np.zeros(n_size)
    
    def get_min(self):
        kara = []
        #print(self.windows)
        #print("="*100)
        a = len(self.windows[0])
        for i in range(len(self.windows)):
          print(i)

          kara.append(np.hstack(self.windows[i]))
        #print(np.vstack(kara).shape)
        return np.min(np.vstack(kara), axis=0).reshape(a, -1) if self.n else np.zeros(n_size)

    def get_maxmin(self):
        kara = []
        a = len(self.windows[0]) #  ここはasetのかず14
        
        #print(a)
        for i in range(len(self.windows)): # ここは全日数の話          
          kara.append(np.hstack(self.windows[i]))
        last_a = np.vstack(kara)

        maxs = np.max(last_a, axis=0).reshape(a, -1) 
        mins =np.min(last_a, axis=0).reshape(a, -1)
        return (maxs[:,1] - mins[:,2]).reshape(a, -1)  if self.n else np.zeros(n_size) # ここ用chk
        # (np.transpose(np.array([HperL, GK_vol, RS_vol,log_r, mpower, base, baseh, basem, Dollars, Volume_per_trade, Dollars_per_trade])))

    def get_realvol(self):
        if len(self.windows) < 15:
          a = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          return np.concatenate((a, a), axis=1) if self.n else np.zeros(n_size)

        # Features使う
        vola = 3
        a = len(self.windows[0])
        #print(a)
        band = np.arange(3, (len(self.windows))*6, 6) # ここ帰る
        b = np.hstack(self.windows)
        c =  np.sqrt(np.nansum(b[:, band[-16:-1]], axis=1)).astype(float).reshape(a, -1)
        #print(c.shape)
        if len(self.windows) < 240:
          #print(xxx)
          d = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          return np.concatenate((c, d), axis=1) if self.n else np.zeros(n_size)
        
        else:
          d =  np.sqrt(np.nansum(b[:, band], axis=1)).astype(float).reshape(a, -1)
          return np.concatenate((c, d), axis=1) if self.n else np.zeros(n_size)

    def get_lag(self):
        # basic features使う
        a = len(self.windows[0])
        #print(len(self.windows))
        band = np.arange(0, (len(self.windows))*9, 9) #np.arange(0, (2)*8, 8) ここ帰る
        #print(self.windows[0].shape)
        b = np.hstack(self.windows)
        b = b[:, [band[0],band[-1]]]
        return np.log(b[:,-1]/b[:,0]).reshape(a, -1) if self.n else np.zeros(n_size)
    
    def get_std(self):
        # Features使う
        a = len(self.windows[0])
        #print(a)
        band = np.arange(5, (len(self.windows))*6, 6) # ここ帰る
        #print(self.windows[0].shape)
        b = np.hstack(self.windows)
        b = b[:, band]
        return np.nanstd(b, axis=1).reshape(a, -1) if self.n else np.zeros(n_size)
    
    def rich_cldiff(self):

        a = len(self.windows[0])
        #print(a)
        band1 = np.arange(6, (len(self.windows))*9 ,9) # ここ帰る

        b = np.hstack(self.windows)
        c = b[:, [band1[0],band1[-1]]]
        diff_log = (c[:,-1] - c[:,0]).reshape(a, -1)
        mean_diff_log = np.nanmean(diff_log)
        mean_diff_log = np.expand_dims(np.array([mean_diff_log for i in range(14)]),axis=1)
        
        band2 = np.arange(7, (len(self.windows))*9, 9) 
        inv_weight_sum = 1 / np.nansum(b[:, band2[-1]])
        #print(inv_weight_sum)
        feature_cl_diff1_mean_w = np.nansum(diff_log * b[:, band2[-1]])* inv_weight_sum
        feature_cl_diff1_mean_w = np.expand_dims(np.array([feature_cl_diff1_mean_w for i in range(14)]),axis=1)

        market_return_causal = np.nansum(diff_log* b[:, band2[-1]]) * inv_weight_sum
        market_return_causal = np.expand_dims(np.array([market_return_causal for i in range(14)]),axis=1)
        #print(market_return_causal)
        feature_cl_diff1_rank = np.argsort(np.argsort(np.hstack(diff_log))).reshape(a, -1)
        #print(feature_cl_diff1_rank)
        #return diff_log, mean_diff_log, inv_weight_sum,  feature_cl_diff1_mean_w, feature_cl_diff1_rank if self.n else np.zeros(n_size)
        return (np.concatenate(((market_return_causal * diff_log), market_return_causal), axis=1), # rich feature 2個とcolse
                   np.concatenate((diff_log, mean_diff_log, feature_cl_diff1_mean_w, feature_cl_diff1_rank),axis=1)
                   if self.n else np.zeros(n_size) )# 'feature_cl_diff1', 'feature_cl_diff1_mean_simple', feature_cl_diff1_mean_simple', 'feature_cl_diff1_rank'

    def rich_beta(self):
        # (np.concatenate((beta_calc_features, baseinfo_features[:,0].reshape(14,1)), axis=1))
        a = len(self.windows[0])
  
        b = np.hstack(self.windows)
        #band1 = np.arange(0, (len(self.windows))*3, 3)
        #band2 = np.arange(1, (len(self.windows))*3, 3)
        # MAfeatures1
        #band3 = np.arange(2, (len(self.windows))*3 ,3)          
        ##############################################
        # MA_Features2 enveroope
        if len(self.windows) < 16:
          
          env_up = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          env_low = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          #chouki = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          #print(env_low)

        else:
          env_std = np.nanstd(b[:, -16:-1], axis=1)
          env_mean = np.nanmean(b[:, -16:-1], axis=1)
          env_up = ((env_mean + env_std - b[:, -1]) / b[:, -1]).reshape(a, -1)
          env_low = ((env_mean + (3*env_std) - b[:, -1]) / b[:, -1]).reshape(a, -1)
        
        #######################################################

       

        return (np.concatenate((env_up, env_low),axis=1) if self.n else np.zeros(n_size))# 'feature_cl_diff1', 'feature_cl_diff1_mean_simple', feature_cl_diff1_mean_simple', 'featu

    ##############################################
    def make_rsi(self):
        a = len(self.windows[0])
        #b = np.hstack(self.windows)

    # RSI feature
        if len(self.windows) < 240 * 15:
          #print(len(self.windows))
          
          rsi15 = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          rsi240 = rsi15#np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          #print(self.windows[-1])
          #print("="*100)
        
        else:
          #baseline = b[:, band3[-15*15]]
          #cols = [(i+1) * 14 for i in range(15)]
          cols2 = [(i+1) * -15 for i in range(14)]
          cols2 = cols2 + [-1]

          #cols = [(i+1) * 240 for i in range(15)]
          cols3 = [(i+1) * -240 for i in range(14)]
          cols3= cols3 + [-1]

          cols4 = [(i+1) * -60 for i in range(14)]
          cols4= cols3 + [-1]
          
          b15 = []
          b240 = []
          #b60 = []
          for i in range(len(cols2)):
            #print(cols2[i])
            #print(len(self.windows))
            b15.append(self.windows[cols2[i]])
            b240.append(self.windows[cols3[i]])
            #b60.append(self.windows[cols4[i]])
            
          

          #b15 = b[:, cols2]
          b15 = np.hstack(b15)
          b240 = np.hstack(b240)
          #b60 = np.hstack(b60)
          #print('A')
          #b15 = b15[:, cols]
          #print(b15)

          #print(np.diff(b15, axis=1))
          diff_15 = np.diff(b15, axis=1)
          diff_240 = np.diff(b240, axis=1)
          #zeros = np.expand_dims(np.array([0 for i in range(14)]),axis=1)

          #print(np.where(diff_15 > 0, 0, diff_15))
          #print(diff_15)
          g = np.sum(np.where(diff_15 < 0, 0, diff_15), axis=1)
          h = abs(np.sum(np.where(diff_15 > 0, 0, diff_15), axis=1))
          i = np.sum(np.where(diff_240 < 0, 0, diff_240), axis=1)
          j = abs(np.sum(np.where(diff_240 > 0, 0, diff_240), axis=1))

          #print(g)
          rsi15 = 100 * (g / (g + h)).reshape(a, -1)
          rsi240 = 100 * (i / (i + j)).reshape(a, -1)
         # macd
        if len(self.windows) < 60 * 27:
          macd15 = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          macd60 = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)

        else:
          #baseline = b[:, band3[-15*15]]
          #cols = [(i+1) * 14 for i in range(15)]
          cols2 = [(i+1) * -15 for i in range(26)]
          cols2 = cols2 + [-1]

          #cols = [(i+1) * 240 for i in range(15)]
          cols3 = [(i+1) * -60 for i in range(26)]
          cols3= cols3 + [-1]
            
          #c15 = b[:, cols2]
          #c60 = b[:, cols3]
          
          c15 = []
          c60 = []
          for i in range(len(cols2)):
            #print(cols2[i])
            #print(len(self.windows))
            c15.append(self.windows[cols2[i]])
            c60.append(self.windows[cols3[i]])

          c15 = np.hstack(c15)
          c60 = np.hstack(c60)
          
          macd15  = (np.nanmean(c15[:, -12:-1],axis=1) - np.nanmean(c15[:, -26:-1],axis=1)).reshape(a, -1)
          macd60  = (np.nanmean(c60[:, -12:-1],axis=1) - np.nanmean(c60[:, -26:-1],axis=1)).reshape(a, -1)
        
        return np.concatenate((macd15, macd60), axis=1), (np.concatenate((rsi15, rsi240), axis=1) if self.n else np.zeros(n_size))
    
    def macd(self):
      if len(self.windows) < 9:
        
        signal15 = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
        signal60 = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
      
      else:
        band1 = np.arange(0, (len(self.windows))*2, 2)
        band2 = np.arange(1, (len(self.windows))*2, 2)

        a = len(self.windows[0])
        b = np.hstack(self.windows)

        signal15 = np.nanmean(b[:, band1], axis=1).reshape(a, -1)
        signal60 = np.nanmean(b[:, band2], axis=1).reshape(a, -1)

        #hist15 = (b[:, -2].reshape(a, -1) - signal15).reshape(a, -1)
        #hist60 = (b[:, -1].reshape(a, -1) - signal60).reshape(a, -1)

        #mtanki = (signal15 > b[:, -2].reshape(a, -1)).astype(int).reshape(a,1)
        #mchouki= (signal60 > b[:, -1].reshape(a, -1)).astype(int).reshape(a,1)

      return (np.concatenate((signal15, signal60), axis=1) if self.n else np.zeros(n_size))

    def vol_sum_std(self):
        #band1 = np.arange(0, (len(self.windows))*7, 7)
        band1 = np.arange(0, (len(self.windows))*2, 2)
        a = len(self.windows[0])
        if len(self.windows) < 16:
          a = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
          return np.concatenate((a,a), axis=1)

        else:
          b = np.hstack(self.windows)
          c = np.nansum(b[:,band1[-16:-1]], axis=1).reshape(a,1)
          d = np.nansum(b[:,band1[-241:-1]], axis=1).reshape(a,1)

          #e = np.nanstd(b[:,band1[-16:-1]], axis=1).reshape(a,1)
          #f = np.nanstd(b[:,band1[-241:-1]], axis=1).reshape(a,1)
        
          #print(upper, upper.shape)
          #print(downer, downer.shape)
          return  np.concatenate((c,d), axis=1)if self.n else np.zeros(n_size)
    

    def mmean_std(self):
      band1 = np.arange(0, (len(self.windows))*2, 2)
      band2 = np.arange(1, (len(self.windows))*2, 2)
      a = len(self.windows[0])
      if len(self.windows) < 240:
        a = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
        return np.concatenate((a, a, a), axis=1)  if self.n else np.zeros(n_size)

      else:
        b = np.vstack(self.windows)
        #print(b)

        c = np.nanmean(b[-16:-1, 0])#.reshape(a,1)
        #print(c)
        d = np.nanmean(b[-241:-1, 0])
        #print(d)
        #e = np.nanmean(b[-16:-1, 1])
        #print(3)
        #f = np.nanmean(b[-241:-1, 1])
        
        g = np.nanstd(b[-16:-1, 0])
        #h = np.nanstd(b[-241:-1, 0])
        #i = np.nanstd(b[-16:-1, 1])
        #j = np.nanstd(b[-241:-1, 1])
       
        ##k = np.nansum(b[-16:-1, 0])
        #l = np.nansum(b[-241:-1, 0])
        #m = np.nansum(b[-16:-1, 1])
        #n = np.nansum(b[-241:-1, 1])

        #o = np.nanmean(b[-1, 0])
        #p = np.nanmean(b[-1, 1])
        #q = np.nansum(b[-1, 0])
        #r = np.nansum(b[-1, 1])
        xx = (np.array([c, d, g]))
        #fin = np.vstack(np.array([xx for i in range(14)]))
        fin = np.vstack(np.array([xx for i in range(14)]))
        #print(fin)
        return fin if self.n else np.zeros(n_size)

    def vol_features(self):
      band1 = np.arange(4, (len(self.windows))*9, 9) # vol
      band2 = np.arange(5, (len(self.windows))*9, 9) # count
      band3 = np.arange(0, (len(self.windows))*9, 9) # close

      a = len(self.windows[0])
      if len(self.windows) < 240:
        a = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
        return np.concatenate((a, a), axis=1)  if self.n else np.zeros(n_size)
        #return a if self.n else np.zeros(n_size)

      else:
        b = np.hstack(self.windows)
        #print(b)
        bs15 = np.max(b[:,band1[-16:-1]], axis=1).reshape(a,1)
        bs240 = np.max(b[:,band1[-240:-1]], axis=1).reshape(a,1)

        bcs15 = b[:,band3[-16]].reshape(a,1)
        bcs240 = b[:,band3[-240]].reshape(a,1)

        #print(bs15)
        #print('='*100)
        #print(b[:,band1[-16:-1]] )
        #print('='*100)
        #print(b[:,band1[-16:-1]] / bs15)

        #b15sum = (np.nansum((b[:,band1[-16:-1]] / bs15), axis=1).reshape(a,1))
        #b240sum = (np.nansum((b[:,band1[-240:-1]] / bs240), axis=1).reshape(a,1))
        #b15mean = (np.nanmean((b[:,band1[-16:-1]] / bs15), axis=1).reshape(a,1))
        #b240mean = (np.nanmean((b[:,band1[-240:-1]] / bs240), axis=1).reshape(a,1))
        #b15std = np.nanstd((b[:,band1[-16:-1]]/ bs15), axis=1).reshape(a,1)
        b240std = np.nanstd((b[:,band1[-240:-1]] / bs240), axis=1).reshape(a,1)

        cs15 = np.max(b[:,band2[-16:-1]], axis=1).reshape(a,1)
        cs240 = np.max(b[:,band2[-240:-1]], axis=1).reshape(a,1)

        #c15sum = (np.nansum((b[:,band2[-16:-1]] / cs15), axis=1).reshape(a,1))
        c240sum = (np.nansum((b[:,band2[-240:-1]] / cs240), axis=1).reshape(a,1))
        #c15mean = (np.nanmean((b[:,band2[-16:-1]] / cs15), axis=1).reshape(a,1))
        #c240mean = (np.nanmean((b[:,band2[-240:-1]] / cs240), axis=1).reshape(a,1))
        #c15std = np.nanstd((b[:,band2[-16:-1]]/ cs15), axis=1).reshape(a,1)
        #c240std = np.nanstd((b[:,band2[-240:-1]] / cs240), axis=1).reshape(a,1)

        #d15sum = (np.nansum(((b[:,band1[-16:-1]] / bs15) / (b[:,band2[-16:-1]] / cs15)), axis=1).reshape(a,1))
        #d240sum = (np.nansum(((b[:,band1[-240:-1]] / bs15) / (b[:,band2[-240:-1]] / cs240)), axis=1).reshape(a,1))
        #d15mean = (np.nanmean(((b[:,band1[-16:-1]] / bs15) / (b[:,band2[-16:-1]] / cs15)), axis=1).reshape(a,1))
        #d240mean = (np.nanmean(((b[:,band1[-240:-1]] / bs15) / (b[:,band2[-240:-1]] / cs240)), axis=1).reshape(a,1))
        #d15std = np.nanstd(((b[:,band1[-16:-1]] / bs15) / (b[:,band2[-16:-1]]/ cs15)), axis=1).reshape(a,1)
        #d240std = np.nanstd(((b[:,band1[-240:-1]] / bs15) / (b[:,band2[-240:-1]] / cs240)), axis=1).reshape(a,1)

        #print(b15sum)
        #e15 = (np.nansum(b[:,band3[-16:-1]] / bcs15, axis=1).reshape(a,1) * b15sum).reshape(a,1)
        #e240 = (np.nansum(b[:,band3[-240:-1]] / bcs240, axis=1).reshape(a,1) * b240sum).reshape(a,1)
        #print()
        

        return np.concatenate((b240std, c240sum), axis=1) if self.n else np.zeros(n_size)

    def oth_teq(self):
      #(tmp_df[["EndOfDayQuote PreviousClose", "EndOfDayQuote High"]].max(axis=1) - tmp_df[["EndOfDayQuote PreviousClose", "EndOfDayQuote Low"]].min(axis=1)) / tmp_df["EndOfDayQuote PreviousClose"]
      #tmp_df = tmp_df.replace([np.inf, -np.inf], np.nan)
      band1 = np.arange(7, (len(self.windows))*9, 9) # open
      band2 = np.arange(1, (len(self.windows))*9, 9) # high
      band3 = np.arange(2, (len(self.windows))*9, 9) # min
      band4 = np.arange(5, (len(self.windows))*9, 9) # vol
      band5 = np.arange(0, (len(self.windows))*9, 9) # close
      a = len(self.windows[0])

      if len(self.windows) < 240:
        a = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
        return np.concatenate((a, a, a, a), axis=1)  if self.n else np.zeros(n_size)

      else:
        b = np.hstack(self.windows)
        #print(b)
        #print(np.max(b[:,band2[-16:-1]], axis=1).reshape(a,1), np.min(b[:,band3[-16:-1]], axis=1).reshape(a,1), b[:,band1[-1]].reshape(a,1))
        #print("="*100)
        bs15 = ((np.nanmax(b[:,band2[-16:-1]], axis=1).reshape(a,1) - np.nanmin(b[:,band3[-16:-1]], axis=1).reshape(a,1)) / b[:,band1[-1]].reshape(a,1))
        bs240 = ((np.nanmax(b[:,band2[-240:-1]], axis=1).reshape(a,1) - np.nanmin(b[:,band3[-240:-1]], axis=1).reshape(a,1)) / b[:,band1[-1]].reshape(a,1))

        #print((np.sum(b[:, band4[-16:1]], axis=1) ))
        mi15 = (bs15 / (np.nansum(b[:, band4[-16:-1]], axis=1) * np.nanmean(b[:, band5[-16:-1]], axis=1)).reshape(a,1)).reshape(a,1)
        mi240 = (bs240 / (np.nansum(b[:, band4[-240:-1]], axis=1) * np.nanmean(b[:, band5[-240:-1]], axis=1)).reshape(a,1)).reshape(a,1)

        return np.concatenate((bs15, bs240, mi15, mi240), axis=1) if self.n else np.zeros(n_size)

    def tec_lags(self):
      #(tmp_df[["EndOfDayQuote PreviousClose", "EndOfDayQuote High"]].max(axis=1) - tmp_df[["EndOfDayQuote PreviousClose", "EndOfDayQuote Low"]].min(axis=1)) / tmp_df["EndOfDayQuote PreviousClose"]
      #tmp_df = tmp_df.replace([np.inf, -np.inf2, np.nan)
      band1 = np.arange(0, (len(self.windows))*9, 9) # open
      band2 = np.arange(1, (len(self.windows))*9, 9) # high
      band3 = np.arange(2, (len(self.windows))*9, 9) # min
      band4 = np.arange(3, (len(self.windows))*9, 9) # vol
      band5 = np.arange(4, (len(self.windows))*9, 9) # close
      band6 = np.arange(5, (len(self.windows))*9, 9) 
      band7 = np.arange(6, (len(self.windows))*9, 9) 
      band8 = np.arange(7, (len(self.windows))*9, 9) 
      band9 = np.arange(8, (len(self.windows))*9, 9) 
      band10 = np.arange(9, (len(self.windows))*9, 9) 

      a = len(self.windows[0])

      if len(self.windows) < 5:
        a = np.expand_dims(np.array([np.nan for i in range(14)]),axis=1)
        return np.concatenate((a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a), axis=1)  if self.n else np.zeros(n_size)

      else:
        b = np.hstack(self.windows)

        l11 = b[:, band1[-1]].reshape(a, -1)
        l12 = b[:, band2[-1]].reshape(a, -1)
        l13 = b[:, band3[-1]].reshape(a, -1)
        l14 = b[:, band4[-1]].reshape(a, -1)
        l15 = b[:, band5[-1]].reshape(a, -1)
        l16 = b[:, band6[-1]].reshape(a, -1)
        l17 = b[:, band7[-1]].reshape(a, -1)
        l18 = b[:, band8[-1]].reshape(a, -1)
        l19 = b[:, band9[-1]].reshape(a, -1)
        l110 = b[:, band10[-1]].reshape(a, -1)

        l21 = b[:, band1[-2]].reshape(a, -1)
        l22 = b[:, band2[-2]].reshape(a, -1)
        l23 = b[:, band3[-2]].reshape(a, -1)
        l24 = b[:, band4[-2]].reshape(a, -1)
        l25 = b[:, band5[-2]].reshape(a, -1)
        l26= b[:, band6[-2]].reshape(a, -1)
        l27 = b[:, band7[-2]].reshape(a, -1)
        l28 = b[:, band8[-2]].reshape(a, -1)
        l29 = b[:, band9[-2]].reshape(a, -1)
        #l210 = b[:, band10[-2]].reshape(a, -1)

        #l31 = b[:, band1[-3]].reshape(a, -1)
        #l32 = b[:, band2[-3]].reshape(a, -1)
        #l33 = b[:, band3[-3]].reshape(a, -1)
        #l34 = b[:, band4[-3]].reshape(a, -1)
        #l35 = b[:, band5[-3]].reshape(a, -1)
        #l36= b[:, band6[-3]].reshape(a, -1)
        #l37 = b[:, band7[-3]].reshape(a, -1)
        #l38 = b[:, band8[-3]].reshape(a, -1)
        #l39 = b[:, band9[-3]].reshape(a, -1)
        #l310 = b[:, band10[-3]].reshape(a, -1)

        #l41 = b[:, band1[-4]].reshape(a, -1)
        #l42 = b[:, band2[-4]].reshape(a, -1)
        #l43 = b[:, band3[-4]].reshape(a, -1)
        #l44 = b[:, band4[-4]].reshape(a, -1)
        #l45 = b[:, band5[-4]].reshape(a, -1)
        #l46= b[:, band6[-4]].reshape(a, -1)
        #l47 = b[:, band7[-4]].reshape(a, -1)
        #l48 = b[:, band8[-4]].reshape(a, -1)
        #l49 = b[:, band9[-4]].reshape(a, -1)
        #l410 = b[:, band10[-4]].reshape(a, -1)



        return np.concatenate((l11,l12,l13,l14,l15,l16,l17,l18,l19,l110, l21,l22,l23,l24,l25,l26,l27,l28,l29,#l210, 
                                           #l31,l32,l33,l34,l35,l36,l37,l38,l39,l310, # l41,l42,l43,l44,l45,l46,l47,l48,l49,l410
                              ), axis=1) if self.n else np.zeros(n_size)

    def __str__(self):
        return "Current window values: {}".format(list(self.windows))

# Temporary removing njit as it cause many bugs down the line
# Problems mainly due to data types, I have to find where I need to constraint types so as not to make njit angry

def fillna_npwhere(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

def Clean_df(x):
    Asset_ID = x[:,1]
    timestamp = x[0,0]

    if len(Asset_ID)<14:
        missing_ID = [i for i in range(14) if i not in Asset_ID]
        for i in missing_ID:
            row = np.array((timestamp,i,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan))
            x = np.concatenate((x,np.expand_dims(row,axis=0)))

    x = x[np.argsort(x[:,1])]
    return (x[:,i] for i in range(x.shape[1]))

def Base_Feature_fn(timestamp,Asset_ID,Count,O,H,L,C,Volume,VWAP):

    VWAP = np.where(np.isinf(VWAP),(C+O)/2,VWAP)
    base = C
    baseh = H                        # 追加
    basem = L
    baseo = O
    HL = H/L
    HperL = base / baseo
    
    O = O/base
    H = H/base
    L = L/base
    C = C/base
    VWAP = VWAP/C
    Price = base

    Dollars = Volume * Price
    Volume_per_trade = Volume/Count
    Dollars_per_trade = Dollars/Count

    # 大口が買った (volume / count) * HL
    mpower = HL#HL * (Volume / Count) # ここはHigh / lowに変更

    log_ret = np.log(base/baseo) # 正しいターゲット
    log_r = (np.log(Price) - np.log(baseo)) ** 2 # ralized vol計算用
    GK_vol = (1 / 2 * np.log(H/L) ** 2 - (2 * np.log(2) - 1) * np.log(C/O) ** 2)
    RS_vol = np.log(H/C)*np.log(H/O) + np.log(L/C)*np.log(L/O)
    
    # uki feature
    #rang = baseh - basem
    #hig_rang = ((baseh - basem) - np.abs(baseo - base)) / baseo
    #mi = rang / Dollars

    #return(np.transpose(np.array([Count,O,H,L,C,Price,Volume,VWAP,Dollars,Volume_per_trade,Dollars_per_trade,log_ret,GK_vol,RS_vol])))
    #return(np.transpose(np.array([HL,Price, VWAP,Dollars,Volume_per_trade,Dollars_per_trade,log_ret,GK_vol,RS_vol,rang,hig_rang, mi])))
    return(np.transpose(np.array([HperL, GK_vol, RS_vol,log_r, mpower, log_ret, base, baseh, basem, Dollars, Volume, Count,log_ret, baseo])))

def Time_Feature_fn(timestamp):
    
    sin_month = (np.sin(2 * np.pi * timestamp.month/12))
    cos_month = (np.cos(2 * np.pi * timestamp.month/12))
    sin_day = (np.sin(2 * np.pi * timestamp.day/31))
    cos_day = (np.cos(2 * np.pi * timestamp.day/31))
    sin_hour = (np.sin(2 * np.pi * timestamp.hour/24))
    cos_hour = (np.cos(2 * np.pi * timestamp.hour/24))
    sin_minute = (np.sin(2 * np.pi * timestamp.minute/60))
    cos_minute = (np.cos(2 * np.pi * timestamp.minute/60))

    return(np.array((sin_month,cos_month,sin_day,cos_day,sin_hour,cos_hour,sin_minute,cos_minute)))

In [13]:
if RETRAIN == False:
    # read dict
    
    dict_RM = pd.read_pickle(os.path.join(savepath, '26_dict_RM.pkl'))
    dict_RM_base = pd.read_pickle(os.path.join(savepath, '26_dict_RM_base.pkl'))
    dict_RM_richbeta = pd.read_pickle(os.path.join(savepath, '26_dict_RM_richbeta.pkl'))
    dict_RM_macd = pd.read_pickle(os.path.join(savepath, '26_dict_RM_macd.pkl'))
    dict_RM_vols = pd.read_pickle(os.path.join(savepath, '26_dict_RM_vols.pkl'))
    dict_RM_market = pd.read_pickle(os.path.join(savepath, '26_dict_RM_market.pkl'))
    dict_RM_lag = pd.read_pickle(os.path.join(savepath, '26_dict_RM_lag.pkl'))
    dict_RM_17 = pd.read_pickle(os.path.join(savepath, '26_dict_RM_17.pkl'))

    #models = [pd.read_pickle(i) for i in glob.glob('../input/gresearch-35/lgbm__lightgbm.basic.Booster object at 0x7f9d23edd650_.pkl/*')]
    #models = pd.read_pickle('../input/gresearch-37-est300/37_est300_lgbm__lightgbm.basic.Booster object at 0x7f3c3104cb90_.pkl')

In [14]:
Features_names = [#'Count','O',
                 # 'HL',
                  'HperL', #'Price',#'Volume',
                 # 'VWAP','Dollars','Volume_per_trade','Dollars_per_trade',
                  #'GK_vol','RS_vol', 'log_r', 'mpower', 'log_ret']#,'hig_rang','mi'
                  'log_r', 'log_ret']#,'hig_rang','mi']
base_names = ['Close', 'High', 'Low', 'Dollars', 'Volume_per_trade', 'Dollars_per_trade']
Market_Features_names = [s+'_M' for s in Features_names]
Time_Features_names = ['sin_month','cos_month','sin_day','cos_day','sin_hour','cos_hour','sin_minute','cos_minute']
MA_Features_names = [s+'_'+str(lag) for lag in MA_lags for s in Features_names ]
MA_Features_M_names = [s+'_'+str(lag) for lag in MA_lags for s in Market_Features_names]
betas_names = ['betas_'+str(lag) for lag in beta_lags]

MA_Featuresmax_names = [s+'_'+str(lag) + '_max' for lag in [15] for s in Features_names ]
MA_Featuresmin_names = [s+'_'+str(lag) + '_min' for lag in [15] for s in Features_names ]
Realized_vol_names = [s+'_'+str(lag)  for lag in [15, 240] for s in ['realized_vol'] ]
MA_Featureslag_names = [s+'_'+str(lag) + '_lag' for lag in [15] for s in ['log_ret']]
MA_Featuresstd_names = [s+'_'+str(lag) + '_std' for lag in [15] for s in ['log_ret']]

Richman_Features_beta_name = [ 'ENV_up', 'ENV_do', 'RSI15', 'RSI240']
macd_Features_name = ['macd60', 'signal15', 'signal60']
#Btc_eth_corr_Features_name = ['Btc_corr', 'Eth_corr']
#Updorate_name = ['Uprate_15', 'Dorate_15', 'Evenrate_15']
Realized_vol_sum_names = [s+'_'+str(lag)  for lag in ['sum_15', 'sum_240'] for s in ['realized_vol_std'] ]
market_Features_names =  ['m15_15mean', 'm15_240mean', 'm15_15std']
vol_Features_names = ['vol240std', 'c240sum']

lag_features_select = ['log_ret'] +  Realized_vol_names + Realized_vol_sum_names  + market_Features_names + vol_Features_names
lag_features_select2 = [s + lag for lag in ['_lag1', '_lag2'] for s in lag_features_select]
#oth_Features_name = ['hl_15', 'hl_240', 'mi15', 'mi240']

All_names = (Features_names + Realized_vol_names  + MA_Featureslag_names +
                    MA_Featuresstd_names + #Richman_Features_name + 
                    Richman_Features_beta_name + 
                    macd_Features_name + Realized_vol_sum_names +  market_Features_names + 
                    vol_Features_names + lag_features_select2) #+ oth_Features_name)#+ #Btc_eth_corr_Features_name + Updorate_name #+ MA_Featuresmaxmin_names #+ Market_Features_names + Time_Features_names + MA_Features_names + MA_Features_M_names + betas_names + MA_Featuresmax_names + MA_Featuresmin_names + Realized_vol_names

All_names.remove('c240sum_lag2')
print(len(All_names))
print(All_names)

40
['HperL', 'log_r', 'log_ret', 'realized_vol_15', 'realized_vol_240', 'log_ret_15_lag', 'log_ret_15_std', 'ENV_up', 'ENV_do', 'RSI15', 'RSI240', 'macd60', 'signal15', 'signal60', 'realized_vol_std_sum_15', 'realized_vol_std_sum_240', 'm15_15mean', 'm15_240mean', 'm15_15std', 'vol240std', 'c240sum', 'log_ret_lag1', 'realized_vol_15_lag1', 'realized_vol_240_lag1', 'realized_vol_std_sum_15_lag1', 'realized_vol_std_sum_240_lag1', 'm15_15mean_lag1', 'm15_240mean_lag1', 'm15_15std_lag1', 'vol240std_lag1', 'c240sum_lag1', 'log_ret_lag2', 'realized_vol_15_lag2', 'realized_vol_240_lag2', 'realized_vol_std_sum_15_lag2', 'realized_vol_std_sum_240_lag2', 'm15_15mean_lag2', 'm15_240mean_lag2', 'm15_15std_lag2', 'vol240std_lag2']


In [15]:
colabcols = [
             'Asset_ID',
             'HperL',
             'log_r',
             'log_ret',
             'realized_vol_15',
             'realized_vol_240',
             'log_ret_15_lag',
             'log_ret_15_std',
             'ENV_up',
             'ENV_do',
             'RSI15',
             'RSI240',
             'macd60',
             'signal15',
             'signal60',
             'realized_vol_std_sum_15',
             'realized_vol_std_sum_240',
             'm15_15mean',
             'm15_240mean',
             'm15_15std',
             'vol240std',
             'c240sum',
             'log_ret_lag1',
             'realized_vol_15_lag1',
             'realized_vol_240_lag1',
             'realized_vol_std_sum_15_lag1',
             'realized_vol_std_sum_240_lag1',
             'm15_15mean_lag1',
             'm15_240mean_lag1',
             'm15_15std_lag1',
             'vol240std_lag1',
             'c240sum_lag1',
             'log_ret_lag2',
             'realized_vol_15_lag2',
             'realized_vol_240_lag2',
             'realized_vol_std_sum_15_lag2',
             'realized_vol_std_sum_240_lag2',
             'm15_15mean_lag2',
             'm15_240mean_lag2',
             'm15_15std_lag2',
             'vol240std_lag2'
]

colabcols_ridge = [
                    #'Asset_ID',
                    'log_r',
                    'log_ret',
                    'realized_vol_15',
                    'realized_vol_240',
                    'log_ret_15_lag',
                    'ENV_up',
                    'ENV_do',
                    'm15_15mean',
                    'm15_240mean',
                    'm15_15std',
                    'log_ret_lag1',
                    'realized_vol_15_lag1',
                    'realized_vol_240_lag1',
                    'realized_vol_std_sum_15_lag1',
                    'realized_vol_std_sum_240_lag1',
                    'm15_15mean_lag1',
                    'm15_240mean_lag1',
                    'm15_15std_lag1',
                    'vol240std_lag1',
                    'log_ret_lag2',
                    'realized_vol_15_lag2',
                    'realized_vol_240_lag2',
                    'realized_vol_std_sum_15_lag2',
                    'realized_vol_std_sum_240_lag2',
                    'm15_15mean_lag2',
                    'm15_240mean_lag2',
                    'm15_15std_lag2',
                    'vol240std_lag2'            
]

print(len(colabcols))
print(len(colabcols_ridge))

41
28


In [16]:
#np.random.seed(42)
np.random.seed(42)
for j in range(2):
    for i in range(10):
        print(np.random.rand())

0.3745401188473625
0.9507143064099162
0.7319939418114051
0.5986584841970366
0.15601864044243652
0.15599452033620265
0.05808361216819946
0.8661761457749352
0.6011150117432088
0.7080725777960455
0.020584494295802447
0.9699098521619943
0.8324426408004217
0.21233911067827616
0.18182496720710062
0.18340450985343382
0.3042422429595377
0.5247564316322378
0.43194501864211576
0.2912291401980419


# Make train

In [17]:
%%time
if RETRAIN == True:

    # make train dataset
    import os
    from random import random

    sampling = 1#0.05
    #sampling = 0.8#0.05

    counter = 0
    np.random.seed(42)
    for fold in range(n_fold):

        df_train_fold = pd.DataFrame()
        df_test_fold = pd.DataFrame()

        df_read = pd.read_parquet("df_fold_"+str(fold)+'.parquet')


        #instantiation Moving average features dict
        dict_RM = {}
        dict_RM_M = {}
        dict_RM_base = {}
        dict_RM_richbeta = {}
        dict_RM_macd = {}
        dict_RM_vols = {}
        dict_RM_market = {}
        dict_RM_lag = {}
        dict_RM_17 = {}


        for lag in MA_lags:
            dict_RM[lag] = RunningMean(lag)
            dict_RM_M[lag] = RunningMean(lag)
            dict_RM_base[lag] = RunningMean(lag)
            dict_RM_market[lag] = RunningMean(lag)


        dict_RM_richbeta[3601] = RunningMean(3601)
        dict_RM_macd[9] = RunningMean(9)
        dict_RM_lag[5] = RunningMean(5)
        dict_RM_17[17] = RunningMean(17)

        for lag in [15, 240]:
          dict_RM_vols[lag] = RunningMean(lag)

        #instantiation dict betas
        #dict_MM = {}
        #dict_Mr = {}
        #for lag in beta_lags:
        #    dict_MM[lag] = RunningMean(lag)
        #    dict_Mr[lag] = RunningMean(lag)

        #f = ['timestamp','Asset_ID','Count','Open','High','Low','Close','Volume','VWAP','Target']
        f = ['timestamp','Asset_ID','Count','Open','High','Low','Close','Volume','VWAP','Re_Calc_Target']

        t = df_read['timestamp'].values
        ids, index = np.unique(t, return_index=True)
        Values = df_read[f].values
        splits = np.split(Values, index[1:])
        out = []

        kara = []
        for time_id, x in tqdm(zip(ids.tolist(), splits)):

            timestamp,Asset_ID,Count,O,H,L,C,Volume, VWAP,Target = Clean_df(x)

            Features = Base_Feature_fn(timestamp,Asset_ID,Count,O,H,L,C,Volume,VWAP)
            baseinfo_features = np.delete(Features, np.s_[0:6], axis=1)
            Features = np.delete(Features, np.s_[-8:], axis=1)        

            #removing wieghts when data is missing so that they don't appears in market
            weigthss = np.where(np.isnan(O),O,weigths) # ここ変えた
            #Market_Features = np.nansum(Features*np.expand_dims(weigths,axis=1)/np.nansum(weigths),axis=0)
            #Market_Features = np.tile(Market_Features,(14,1))

            #np.array((sin_month,cos_month,sin_day,cos_day,sin_hour,cos_hour,sin_minute,cos_minute))
            time = timestamp_to_date(timestamp[0])
            Time_Features = Time_Feature_fn(time)
            #Time_Features = np.tile(Time_Features,(14,1))

            MA_Features = []
            #MA_Features_M  = []
            #MA_Features_max = [] # max featur
            #MA_Features_min = [] # min feature
            MA_Features_maxmin = []
            RealV_Feature = []
            RealV_Feature_15240 = []
            last_log_r_Feature = []
            Std_Features = []
            Richman_Features = []
            Richman_Features_beata = []
            Btc_eth_corr_Features = []
            Updo_rate_Features = []
            MACD_Featuees = []
            Market_Features = []
            Volume_Features = []
            Other_teq_Features = []
            teq_lagfeatures = []
            RSI_features = []

            for lag in MA_lags:
                dict_RM[lag].push(Features)
                #dict_RM_M[lag].push(Market_Features)
                dict_RM_base[lag].push(np.concatenate((baseinfo_features, weigthss.reshape(14,1)), axis=1)) # ここ変えた
                #dict_RM_richbeta
                #MA_Features.append(dict_RM[lag].get_mean())
                #MA_Features_M.append(dict_RM_M[lag].get_mean())
                #RealV_Feature.append(dict_RM[lag].get_realvol()) 


            last_log_r_Feature.append(dict_RM_base[15].get_lag()) # ここ
            Std_Features.append(dict_RM[15].get_std())

            beta_calc_features, rich_features = dict_RM_base[15].rich_cldiff() # richman feature1　ここ
            Richman_Features.append(rich_features)


            dict_RM_17[17].push(baseinfo_features[:,0].reshape(14,1))
            rich_tech_fe, macd_fe = dict_RM_17[17].rich_beta()
            Richman_Features_beata.append(rich_tech_fe)
            #print('A')
            dict_RM_richbeta[3601].push(baseinfo_features[:,0].reshape(14,1)) # richman feature2
            RSI_features.append(dict_RM_richbeta[3601].make_rsi())
            #print('A')
            #rich_tech_fe, macd_fe = dict_RM_richbeta[3601].rich_beta()



            #print(Richman_Features_beata)

            dict_RM_macd[9].push(macd_fe)
            #print('A')
            MACD_Featuees.append(np.concatenate((np.delete(macd_fe, np.s_[0], axis=1), dict_RM_macd[9].macd()), axis=1))
            #print('A')

            #Btc_eth_corr_Features.append(dict_RM_base[15].corr_btc_eth())
            #Updo_rate_Features.append(dict_RM_base[15].updown_rate())

            RealV_Feature.append(dict_RM[240].get_realvol())  # vola
            #print('A')

            dict_RM_vols[240].push(dict_RM[15].get_realvol())
            RealV_Feature_15240.append(dict_RM_vols[240].vol_sum_std())

            market_mean = np.nanmean(Features[:,5])
            market_mean_abs = np.nanmean(abs(Features[:,5]))
            dict_RM_market[240].push(np.array([market_mean, market_mean_abs]))
            Market_Features.append(dict_RM_market[240].mmean_std())

            Volume_Features.append(dict_RM_base[240].vol_features())





            #Other_teq_Features.append(dict_RM_base[240].oth_teq())
            #print('zzz')
            #RealV_Feature.append(dict_RM[60].get_realvol())

            #for term in [60, 240, 1440]:
            #  MA_Features_maxmin.append(dict_RM_base[term].get_maxmin())# maxmin
              #print('A')


            #print('A')
            #MA_Features = np.concatenate(MA_Features,axis=1)
            #print('B')
            #MA_Features_M = np.concatenate(MA_Features_M)
            #MA_Features_maxmin = np.concatenate(MA_Features_maxmin,axis=1) # maxmin
            #print('C')

            #MA_Features_max = np.concatenate(MA_Features_max,axis=1)
            #MA_Features_min = np.concatenate(MA_Features_min,axis=1)
            RealV_Feature = np.concatenate(RealV_Feature,axis=1) # vola

            last_log_r_Feature = np.concatenate(last_log_r_Feature,axis=1) # lag
            Std_Features = np.concatenate(Std_Features,axis=1) # 標準偏差

            #Richman_Features = np.concatenate(Richman_Features, axis=1) # richman feature1
            Richman_Features_beata = np.concatenate(Richman_Features_beata, axis=1) # richman feature1
            RSI_features = np.concatenate(RSI_features, axis=1)
            MACD_Featuees = np.concatenate(MACD_Featuees, axis=1)

            RealV_Feature_15240 = np.concatenate(RealV_Feature_15240, axis=1)

            Market_Features = np.concatenate(Market_Features, axis=1)
            Volume_Features = np.concatenate(Volume_Features, axis=1)




            dict_RM_lag[5].push(np.concatenate((np.delete(Features, np.s_[:-1], axis=1),  RealV_Feature, RealV_Feature_15240, Market_Features, Volume_Features), axis=1))
            #print('A')
            teq_lagfeatures.append(dict_RM_lag[5].tec_lags())
            teq_lagfeatures = np.concatenate(teq_lagfeatures, axis=1)

            #Other_teq_Features = np.concatenate(Other_teq_Features, axis=1)
            #break
            #print('A')
            #Btc_eth_corr_Features = np.concatenate(Btc_eth_corr_Features, axis=1)
            #Updo_rate_Features = np.concatenate(Updo_rate_Features, axis=1)


            #print('A')
            #MA_Features_M = np.tile(MA_Features_M,(14,1))
            #print('A')
            betas = []
            #break
            #for lag in beta_lags:
            #    dict_MM[lag].push(Market_Features[1]**2)
            #    dict_Mr[lag].push(Market_Features[1]*Features[:,1]) # ここ注意
            #    betas.append(np.expand_dims(dict_Mr[lag].get_mean()/dict_MM[lag].get_mean(),axis=1))
            #break
            #print('A')
            Features = np.delete(Features, np.s_[1:3], axis=1)
            Features = np.delete(Features, 2, axis=1)
            #print('C')
            #betas = np.concatenate(betas,axis=1)
            #betas = np.nan_to_num(betas, nan=0., posinf=0., neginf=0.) 
            #Features = np.delete(Features, 4, axis=1)
            #values = np.concatenate((Features,np.tile(Market_Features,(14,1)),np.tile(Time_Features,(14,1)),MA_Features,np.tile(MA_Features_M,(14,1)),betas, MA_Features_max, MA_Features_min, RealV_Feature, np.expand_dims(Target,axis=1)),axis=1)
            #values = np.concatenate((Features,RealV_Feature,betas,MA_Features, MA_Features_maxmin,np.expand_dims(Target,axis=1)),axis=1)
            #values = np.concatenate((Features,RealV_Feature,last_log_r_Feature, Std_Features, Richman_Features,Richman_Features_beata, MACD_Featuees, Btc_eth_corr_Features, Updo_rate_Features, np.expand_dims(Target,axis=1)),axis=1)
            values = np.concatenate((Features,RealV_Feature,last_log_r_Feature, Std_Features, #Richman_Features,
                                                  Richman_Features_beata, RSI_features, 
                                                  MACD_Featuees, RealV_Feature_15240, Market_Features, 
                                                  Volume_Features, #Other_teq_Features, 
                                                  teq_lagfeatures, 
                                                  np.expand_dims(Target,axis=1)),axis=1)
            #break
            #print('B')
            if np.random.rand() < sampling:
                out.append(np.concatenate((np.expand_dims(timestamp,axis=1),np.expand_dims(Asset_ID,axis=1),np.float32(values)),axis=1))
        #print(values.shape)
        #print(len(out))
        #df_out = pd.DataFrame(np.concatenate(out), columns = ['timestamp','Asset_ID'] + All_names + ['Target']).astype({'timestamp': 'int64','Asset_ID': 'int64'})
        train = pd.DataFrame(np.concatenate(out), columns = ['timestamp','Asset_ID'] + All_names + ['Target']).astype({'timestamp': 'int64','Asset_ID': 'int64'})
        train = train[~np.isnan(train.Target)]
        #df_out = df_out[~np.isnan(df_out.Target)]
        train.to_parquet('train_fold_'+str(fold)+'.parquet')
        print('before:', peak_memory()/10**6, 'GB')

        #df_test_fold.to_parquet('test_fold_'+str(fold)+'.parquet')
        del train
        gc.collect()
        #ind_train = df_out.timestamp.isin(dict_fold['train_fold_'+str(fold)])
        #ind_test = df_out.timestamp.isin(dict_fold['test_fold_'+str(fold)])

        #df_train_fold = df_out[ind_train]
        #df_test_fold = df_out[ind_test]

        #df_train_fold['fold'] = fold
        #df_test_fold['fold'] = fold
        '''
        if counter == 0:
            del df_read
            gc.collect()
            #print(df_train_fold.head())
            train = df_train_fold.copy()
            del df_train_fold
            gc.collect()

            valid = df_test_fold.copy()
            del df_test_fold
            gc.collect()

            counter += 1
            #print('A')

        else:
            del df_read
            gc.collect()
            #print('B')
            train = pd.concat([train, df_train_fold]).reset_index(drop=True)
            valid = pd.concat([valid, df_test_fold]).reset_index(drop=True)

            del df_train_fold, df_test_fold
            gc.collect()

        #df_train_fold.to_parquet('train_fold_'+str(fold)+'.parquet')
        #df_test_fold.to_parquet('test_fold_'+str(fold)+'.parquet')
        '''

if SUPPLEMENT == True:


    # make train dataset
    import os
    from random import random

    sampling = 1#0.05
    #sampling = 0.8#0.05
    n_fold = 1

    counter = 0
    np.random.seed(42)
    for fold in range(n_fold):

        df_train_fold = pd.DataFrame()
        df_test_fold = pd.DataFrame()

        #df_read = pd.read_parquet("df_fold_"+str(fold)+'.parquet')


        #instantiation dict betas
        #dict_MM = {}
        #dict_Mr = {}
        #for lag in beta_lags:
        #    dict_MM[lag] = RunningMean(lag)
        #    dict_Mr[lag] = RunningMean(lag)

        #f = ['timestamp','Asset_ID','Count','Open','High','Low','Close','Volume','VWAP','Target']
        f = ['timestamp','Asset_ID','Count','Open','High','Low','Close','Volume','VWAP','Target']

        t = df_read['timestamp'].values
        ids, index = np.unique(t, return_index=True)
        Values = df_read[f].values
        splits = np.split(Values, index[1:])
        out = []

        kara = []
        for time_id, x in tqdm(zip(ids.tolist(), splits)):

            timestamp,Asset_ID,Count,O,H,L,C,Volume, VWAP,Target = Clean_df(x)

            Features = Base_Feature_fn(timestamp,Asset_ID,Count,O,H,L,C,Volume,VWAP)
            baseinfo_features = np.delete(Features, np.s_[0:6], axis=1)
            Features = np.delete(Features, np.s_[-8:], axis=1)        

            #removing wieghts when data is missing so that they don't appears in market
            weigthss = np.where(np.isnan(O),O,weigths) # ここ変えた
            #Market_Features = np.nansum(Features*np.expand_dims(weigths,axis=1)/np.nansum(weigths),axis=0)
            #Market_Features = np.tile(Market_Features,(14,1))

            #np.array((sin_month,cos_month,sin_day,cos_day,sin_hour,cos_hour,sin_minute,cos_minute))
            time = timestamp_to_date(timestamp[0])
            Time_Features = Time_Feature_fn(time)
            #Time_Features = np.tile(Time_Features,(14,1))

            MA_Features = []
            #MA_Features_M  = []
            #MA_Features_max = [] # max featur
            #MA_Features_min = [] # min feature
            MA_Features_maxmin = []
            RealV_Feature = []
            RealV_Feature_15240 = []
            last_log_r_Feature = []
            Std_Features = []
            Richman_Features = []
            Richman_Features_beata = []
            Btc_eth_corr_Features = []
            Updo_rate_Features = []
            MACD_Featuees = []
            Market_Features = []
            Volume_Features = []
            Other_teq_Features = []
            teq_lagfeatures = []
            RSI_features = []

            for lag in MA_lags:
                dict_RM[lag].push(Features)
                #dict_RM_M[lag].push(Market_Features)
                dict_RM_base[lag].push(np.concatenate((baseinfo_features, weigthss.reshape(14,1)), axis=1)) # ここ変えた
                #dict_RM_richbeta
                #MA_Features.append(dict_RM[lag].get_mean())
                #MA_Features_M.append(dict_RM_M[lag].get_mean())
                #RealV_Feature.append(dict_RM[lag].get_realvol()) 


            last_log_r_Feature.append(dict_RM_base[15].get_lag()) # ここ
            Std_Features.append(dict_RM[15].get_std())

            beta_calc_features, rich_features = dict_RM_base[15].rich_cldiff() # richman feature1　ここ
            Richman_Features.append(rich_features)


            dict_RM_17[17].push(baseinfo_features[:,0].reshape(14,1))
            rich_tech_fe = dict_RM_17[17].rich_beta()
            Richman_Features_beata.append(rich_tech_fe)
            #print('A')

            dict_RM_richbeta[3601].push(baseinfo_features[:,0].reshape(14,1)) # richman feature2
            macd_fe, rsi_fe = dict_RM_richbeta[3601].make_rsi()
            RSI_features.append(rsi_fe)
            #print('A')
            #rich_tech_fe, macd_fe = dict_RM_richbeta[3601].rich_beta()



            #print(Richman_Features_beata)

            dict_RM_macd[10].push(macd_fe)
            #print('A')
            MACD_Featuees.append(np.concatenate((np.delete(macd_fe, np.s_[0], axis=1), dict_RM_macd[10].macd()), axis=1))
            #print('A')

            #Btc_eth_corr_Features.append(dict_RM_base[15].corr_btc_eth())
            #Updo_rate_Features.append(dict_RM_base[15].updown_rate())

            RealV_Feature.append(dict_RM[240].get_realvol())  # vola
            #print('A')

            dict_RM_vols[240].push(dict_RM[15].get_realvol())
            RealV_Feature_15240.append(dict_RM_vols[240].vol_sum_std())

            market_mean = np.nanmean(Features[:,5])
            market_mean_abs = np.nanmean(abs(Features[:,5]))
            dict_RM_market[240].push(np.array([market_mean, market_mean_abs]))
            Market_Features.append(dict_RM_market[240].mmean_std())

            Volume_Features.append(dict_RM_base[240].vol_features())





            #Other_teq_Features.append(dict_RM_base[240].oth_teq())
            #print('zzz')
            #RealV_Feature.append(dict_RM[60].get_realvol())

            #for term in [60, 240, 1440]:
            #  MA_Features_maxmin.append(dict_RM_base[term].get_maxmin())# maxmin
              #print('A')


            #print('A')
            #MA_Features = np.concatenate(MA_Features,axis=1)
            #print('B')
            #MA_Features_M = np.concatenate(MA_Features_M)
            #MA_Features_maxmin = np.concatenate(MA_Features_maxmin,axis=1) # maxmin
            #print('C')

            #MA_Features_max = np.concatenate(MA_Features_max,axis=1)
            #MA_Features_min = np.concatenate(MA_Features_min,axis=1)
            RealV_Feature = np.concatenate(RealV_Feature,axis=1) # vola

            last_log_r_Feature = np.concatenate(last_log_r_Feature,axis=1) # lag
            Std_Features = np.concatenate(Std_Features,axis=1) # 標準偏差

            #Richman_Features = np.concatenate(Richman_Features, axis=1) # richman feature1
            Richman_Features_beata = np.concatenate(Richman_Features_beata, axis=1) # richman feature1
            RSI_features = np.concatenate(RSI_features, axis=1)
            MACD_Featuees = np.concatenate(MACD_Featuees, axis=1)

            RealV_Feature_15240 = np.concatenate(RealV_Feature_15240, axis=1)

            Market_Features = np.concatenate(Market_Features, axis=1)
            Volume_Features = np.concatenate(Volume_Features, axis=1)




            dict_RM_lag[5].push(np.concatenate((np.delete(Features, np.s_[:-1], axis=1),  RealV_Feature, RealV_Feature_15240, Market_Features, Volume_Features), axis=1))
            #print('A')
            teq_lagfeatures.append(dict_RM_lag[5].tec_lags())
            teq_lagfeatures = np.concatenate(teq_lagfeatures, axis=1)

            #Other_teq_Features = np.concatenate(Other_teq_Features, axis=1)
            #break
            #print('A')
            #Btc_eth_corr_Features = np.concatenate(Btc_eth_corr_Features, axis=1)
            #Updo_rate_Features = np.concatenate(Updo_rate_Features, axis=1)


            #print('A')
            #MA_Features_M = np.tile(MA_Features_M,(14,1))
            #print('A')
            betas = []
            #break
            #for lag in beta_lags:
            #    dict_MM[lag].push(Market_Features[1]**2)
            #    dict_Mr[lag].push(Market_Features[1]*Features[:,1]) # ここ注意
            #    betas.append(np.expand_dims(dict_Mr[lag].get_mean()/dict_MM[lag].get_mean(),axis=1))
            #break
            #print('A')
            Features = np.delete(Features, np.s_[1:3], axis=1)
            Features = np.delete(Features, 2, axis=1)
            #print('C')
            #betas = np.concatenate(betas,axis=1)
            #betas = np.nan_to_num(betas, nan=0., posinf=0., neginf=0.) 
            #Features = np.delete(Features, 4, axis=1)
            #values = np.concatenate((Features,np.tile(Market_Features,(14,1)),np.tile(Time_Features,(14,1)),MA_Features,np.tile(MA_Features_M,(14,1)),betas, MA_Features_max, MA_Features_min, RealV_Feature, np.expand_dims(Target,axis=1)),axis=1)
            #values = np.concatenate((Features,RealV_Feature,betas,MA_Features, MA_Features_maxmin,np.expand_dims(Target,axis=1)),axis=1)
            #values = np.concatenate((Features,RealV_Feature,last_log_r_Feature, Std_Features, Richman_Features,Richman_Features_beata, MACD_Featuees, Btc_eth_corr_Features, Updo_rate_Features, np.expand_dims(Target,axis=1)),axis=1)
            values = np.concatenate((Features,RealV_Feature,last_log_r_Feature, Std_Features, #Richman_Features,
                                                  Richman_Features_beata, RSI_features, 
                                                  MACD_Featuees, RealV_Feature_15240, Market_Features, 
                                                  Volume_Features, #Other_teq_Features, 
                                                  teq_lagfeatures, 
                                                  np.expand_dims(Target,axis=1)),axis=1)
            #break
            #print('B')
            if np.random.rand() < sampling:
                out.append(np.concatenate((np.expand_dims(timestamp,axis=1),np.expand_dims(Asset_ID,axis=1),np.float32(values)),axis=1))
        #print(values.shape)
        #print(len(out))
        #df_out = pd.DataFrame(np.concatenate(out), columns = ['timestamp','Asset_ID'] + All_names + ['Target']).astype({'timestamp': 'int64','Asset_ID': 'int64'})
        train = pd.DataFrame(np.concatenate(out), columns = ['timestamp','Asset_ID'] + All_names + ['Target']).astype({'timestamp': 'int64','Asset_ID': 'int64'})
        train = train[~np.isnan(train.Target)]
        #df_out = df_out[~np.isnan(df_out.Target)]
        train.to_parquet('train_fold_'+str(fold)+'.parquet')
        print('before:', peak_memory()/10**6, 'GB')
        #df_test_fold.to_parquet('test_fold_'+str(fold)+'.parquet')
        del train
        gc.collect()
        #ind_train = df_out.timestamp.isin(dict_fold['train_fold_'+str(fold)])
        #ind_test = df_out.timestamp.isin(dict_fold['test_fold_'+str(fold)])

        #df_train_fold = df_out[ind_train]
        #df_test_fold = df_out[ind_test]

        #df_train_fold['fold'] = fold
        #df_test_fold['fold'] = fold
        '''
        if counter == 0:
            del df_read
            gc.collect()
            #print(df_train_fold.head())
            train = df_train_fold.copy()
            del df_train_fold
            gc.collect()

            valid = df_test_fold.copy()
            del df_test_fold
            gc.collect()

            counter += 1
            #print('A')

        else:
            del df_read
            gc.collect()
            #print('B')
            train = pd.concat([train, df_train_fold]).reset_index(drop=True)
            valid = pd.concat([valid, df_test_fold]).reset_index(drop=True)

            del df_train_fold, df_test_fold
            gc.collect()

        #df_train_fold.to_parquet('train_fold_'+str(fold)+'.parquet')
        #df_test_fold.to_parquet('test_fold_'+str(fold)+'.parquet')
        '''

179996it [14:20, 209.10it/s]


before: 3.969724 GB
CPU times: user 14min 25s, sys: 11.2 s, total: 14min 37s
Wall time: 14min 31s


In [18]:
# メモリ使用率を取得
mem_bytes = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')
mem = psutil.virtual_memory() 
print('Train Reloading保存後')
print(mem.percent)
print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
print('Peak:', peak_memory()/10**6, 'GB')

Train Reloading保存後
13.1
Use Memory total: 2.302899784088135 GiB
Peak: 3.969724 GB


In [19]:
if SUPPLEMENT == True:
    
    for i in tqdm(range(n_fold)):
        if i == 0:
            supples = pd.read_parquet(f'./train_fold_{i}.parquet')
            print(supples.shape)
        else:
            base = pd.read_parquet(f'./train_fold_{i}.parquet')
            #train = train.append(base)
            supples = pd.concat([supples, base], ignore_index=True, copy=False)

            # メモリ使用率を取得

            mem = psutil.virtual_memory() 
            print(mem.percent)
            print(f'Use Memory total before del: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
            print('-'*100)
            #print(train.shape)
            print('before:', peak_memory()/10**6, 'GB')

            del base 
            gc.collect()
            print('-'*100)
            print(mem.percent)
            print(f'Use Memory total after del: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
            print('='*100)
    
    print(pd.to_datetime(supples.head(1)['timestamp'].values[0], unit='s'))
    print(pd.to_datetime(supples.tail(1)['timestamp'].values[0], unit='s'))
    display(supples.head())

100%|██████████| 1/1 [00:02<00:00,  2.60s/it]

(2515355, 43)
2021-09-21 00:01:00
2022-01-23 23:44:00


,timestamp,Asset_ID,HperL,log_r,log_ret,realized_vol_15,realized_vol_240,log_ret_15_lag,log_ret_15_std,ENV_up,...,log_ret_lag2,realized_vol_15_lag2,realized_vol_240_lag2,realized_vol_std_sum_15_lag2,realized_vol_std_sum_240_lag2,m15_15mean_lag2,m15_240mean_lag2,m15_15std_lag2,vol240std_lag2,Target
0,1632182460,0,0.999770,5.270550e-08,-0.000230,0.009701,0.024363,0.004875,0.002283,0.000935,...,0.001677,0.156770,73.849289,-0.000982,0.009697,0.024374,0.124613,1.269582,0.000441,-0.019375
1,1632182460,1,0.999091,8.271903e-07,-0.000909,0.006673,0.020532,0.003660,0.001637,0.001052,...,0.001677,0.111814,48.288921,-0.000173,0.006693,0.020553,0.085028,1.055735,0.000441,-0.000374
2,1632182460,2,0.999475,2.757085e-07,-0.000525,0.008905,0.018004,0.002281,0.002211,0.003926,...,0.001677,0.132563,65.391998,-0.001936,0.008693,0.018004,0.090663,0.871884,0.000441,-0.000607
3,1632182460,3,0.998764,1.529583e-06,-0.001237,0.012784,0.033802,0.001042,0.003119,0.003789,...,0.001677,0.161315,68.010574,-0.000898,0.012774,0.033779,0.134338,1.794582,0.000441,0.003085
4,1632182460,4,0.999396,3.645848e-07,-0.000604,0.011326,0.027561,0.003975,0.002786,0.000483,...,0.001677,0.091856,30.491564,-0.001575,0.011226,0.027559,0.117603,1.413247,0.000441,-0.001233


In [20]:
if RETRAIN == True:
    for i in tqdm(range(5)):
        if i == 0:
            train = pd.read_parquet(f'./train_fold_{i}.parquet')
            print(train.shape)
        else:
            base = pd.read_parquet(f'./train_fold_{i}.parquet')
            #train = train.append(base)
            train = pd.concat([train, base], ignore_index=True, copy=False)

            # メモリ使用率を取得

            mem = psutil.virtual_memory() 
            print(mem.percent)
            print(f'Use Memory total before del: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
            print('-'*100)
            #print(train.shape)
            print('before:', peak_memory()/10**6, 'GB')

            del base 
            gc.collect()
            print('-'*100)
            print(mem.percent)
            print(f'Use Memory total after del: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
            print('='*100)

    print('='*100)
    print('='*100)
    filter_validation = pd.to_datetime(train['timestamp'], unit='s') >= '2021-04-01 00:00:00'
    filter_train = pd.to_datetime(train['timestamp'], unit='s') < '2021-04-01 00:00:00'

    print('='*100)
    mem = psutil.virtual_memory() 
    print('valid作成前')
    print(mem.percent)
    print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
    print('Peak:', peak_memory()/10**6, 'GB')


    valid = train[filter_validation]#
    valid.reset_index(drop=True, inplace=True)
    #valid = train.copy()#[filter_validation].reset_index(drop=True)
    gc.collect()
    mem = psutil.virtual_memory() 
    print('valid作成後')
    print(mem.percent)
    print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
    print('Peak:', peak_memory()/10**6, 'GB')



    train = train[filter_train]
    train.reset_index(drop=True, inplace=True)
    mem = psutil.virtual_memory() 
    print('Train作成gc前')
    print(mem.percent)
    print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
    print('Peak:', peak_memory()/10**6, 'GB')
    gc.collect()

else:
    
    train = pd.read_parquet(os.path.join(savepath, '2020_20210920_d2.parquet'))

    print('='*100)
    filter_validation = pd.to_datetime(train['timestamp'], unit='s') >= '2021-04-01 00:00:00'
    filter_train = pd.to_datetime(train['timestamp'], unit='s') < '2021-04-01 00:00:00'

    print('='*100)
    mem = psutil.virtual_memory() 
    print('valid作成前')
    print(mem.percent)
    print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
    print('Peak:', peak_memory()/10**6, 'GB')

    
    if VALID:
        
        valid = train[filter_validation]#
        valid.reset_index(drop=True, inplace=True)
        #valid = train.copy()#[filter_validation].reset_index(drop=True)
        gc.collect()
        mem = psutil.virtual_memory() 
        print('valid作成後')
        print(mem.percent)
        print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
        print('Peak:', peak_memory()/10**6, 'GB')


        train = train[filter_train]
        train.reset_index(drop=True, inplace=True)
        mem = psutil.virtual_memory() 
        print('Train作成gc前')
        print(mem.percent)
        print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
        print('Peak:', peak_memory()/10**6, 'GB')
        gc.collect()


        filter_leakage = pd.to_datetime(train['timestamp'], unit='s') < '2021-06-13 00:00:00'
        filter_memory = pd.to_datetime(train['timestamp'], unit='s') >= '2019-01-01 00:00:00'

        #if LEAKAGE_FILTER:
        #    train = train[filter_leakage]

        if YEAR_FILTER:
            train = train[filter_memory]
            train.reset_index(drop=True, inplace=True)
            print('Train作成gc前')
            print(mem.percent)
            print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
            print('Peak:', peak_memory()/10**6, 'GB')
            gc.collect()
    
    else:
        pass

valid作成前
39.1
Use Memory total: 6.873540576934815 GiB
Peak: 10.66072 GB


In [21]:
print(pd.to_datetime(train.head(1)['timestamp'].values[0], unit='s'))
print(pd.to_datetime(train.tail(1)['timestamp'].values[0], unit='s'))

2020-01-01 00:01:00
2021-09-20 23:44:00


In [22]:
print(train.shape)
if RESAMPLE:
    train = train.sample(frac=0.59, random_state=91, ignore_index=True)
    train.sort_values('timestamp', inplace=True)
    
    delcount = len(supples) - basesize
    
    train = train.iloc[delcount:, :]
    train.reset_index(drop=True, inplace=True)
    print(train.shape)

(11966420, 43)
(6781327, 43)


In [23]:
# SUPPLEと結合
if SUPPLEMENT == True:
    train = pd.concat([train, supples], ignore_index=True, copy=False)
    del supples
    gc.collect()
    print(train.shape)

(9296682, 43)


In [24]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print('Train Reloadingあと')
print(mem.percent)
print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
print('Peak:', peak_memory()/10**6, 'GB')

Train Reloadingあと
30.0
Use Memory total: 5.273816299438477 GiB
Peak: 10.66072 GB


In [25]:
display(train.head())
print('='*100)
display(train.tail())

print(pd.to_datetime(train.head(1)['timestamp'].values[0], unit='s'))
print(pd.to_datetime(train.tail(1)['timestamp'].values[0], unit='s'))
print('='*100)

,timestamp,Asset_ID,HperL,log_r,log_ret,realized_vol_15,realized_vol_240,log_ret_15_lag,log_ret_15_std,ENV_up,...,log_ret_lag2,realized_vol_15_lag2,realized_vol_240_lag2,realized_vol_std_sum_15_lag2,realized_vol_std_sum_240_lag2,m15_15mean_lag2,m15_240mean_lag2,m15_15std_lag2,vol240std_lag2,Target
0,1580206980,12,1.000477,2.273363e-07,0.000477,0.001020,0.006357,0.001847,0.000276,-0.001096,...,0.000368,0.143691,46.000000,-0.000075,0.001041,0.006339,0.012756,0.368343,0.000269,0.005791
1,1580207040,13,1.000488,2.383707e-07,0.000488,0.004702,0.013493,0.009269,0.001003,-0.001576,...,0.000447,0.146778,35.355247,0.001112,0.004609,0.013489,0.067099,0.740592,0.000300,0.006807
2,1580207040,12,1.000418,1.747403e-07,0.000418,0.001123,0.006365,0.002424,0.000287,-0.001339,...,0.000447,0.143763,45.968586,0.000477,0.001020,0.006357,0.012842,0.366611,0.000300,0.004799
3,1580207040,3,1.002951,8.684016e-06,0.002947,0.005945,0.037642,0.001980,0.001695,0.002283,...,0.000447,0.166585,40.416668,0.001788,0.005677,0.037527,0.078178,1.853841,0.000300,0.001504
4,1580207040,9,1.001632,2.659954e-06,0.001631,0.003315,0.010048,0.009525,0.000797,-0.002839,...,0.000447,0.114858,43.766735,0.000112,0.003315,0.009917,0.038019,0.547132,0.000300,0.003708


,timestamp,Asset_ID,HperL,log_r,log_ret,realized_vol_15,realized_vol_240,log_ret_15_lag,log_ret_15_std,ENV_up,...,log_ret_lag2,realized_vol_15_lag2,realized_vol_240_lag2,realized_vol_std_sum_15_lag2,realized_vol_std_sum_240_lag2,m15_15mean_lag2,m15_240mean_lag2,m15_15std_lag2,vol240std_lag2,Target
9296677,1642981440,9,1.000289,8.367403e-08,0.000289,0.005176,0.028407,0.003167,0.001252,-0.000827,...,0.001378,0.107135,18.330618,0.002649,0.005421,0.028412,0.101516,1.472337,-0.000466,0.000047
9296678,1642981440,10,1.000647,4.182542e-07,0.000647,0.003105,0.023972,-0.002785,0.000798,0.003547,...,0.001378,0.111720,24.812679,0.000285,0.003752,0.023963,0.066806,1.140551,-0.000466,-0.002006
9296679,1642981440,11,0.999680,1.025337e-07,-0.000320,0.006513,0.025811,0.008499,0.001631,-0.000911,...,0.001378,0.117991,15.304606,0.000948,0.007363,0.025817,0.102468,1.314910,-0.000466,0.000406
9296680,1642981440,12,1.001190,1.413690e-06,0.001189,0.005232,0.027100,0.004869,0.001336,-0.000416,...,0.001378,0.091844,26.067522,0.001647,0.007514,0.027074,0.116046,1.383625,-0.000466,-0.001046
9296681,1642981440,13,1.000235,5.532951e-08,0.000235,0.006476,0.022716,-0.004386,0.001632,0.003555,...,0.001378,0.076103,10.416759,0.003125,0.006324,0.022714,0.082288,1.065773,-0.000466,0.000281


2020-01-28 10:23:00
2022-01-23 23:44:00


In [26]:
if VALID:
    display(valid.head())
    print('='*100)
    display(valid.tail())

    print(pd.to_datetime(valid.head(1)['timestamp'].values[0], unit='s'))
    print(pd.to_datetime(valid.tail(1)['timestamp'].values[0], unit='s'))
    print('='*100)

In [27]:
features = colabcols
print(features)
print("="*100)
features_ridge = colabcols_ridge
print(features_ridge)

['Asset_ID', 'HperL', 'log_r', 'log_ret', 'realized_vol_15', 'realized_vol_240', 'log_ret_15_lag', 'log_ret_15_std', 'ENV_up', 'ENV_do', 'RSI15', 'RSI240', 'macd60', 'signal15', 'signal60', 'realized_vol_std_sum_15', 'realized_vol_std_sum_240', 'm15_15mean', 'm15_240mean', 'm15_15std', 'vol240std', 'c240sum', 'log_ret_lag1', 'realized_vol_15_lag1', 'realized_vol_240_lag1', 'realized_vol_std_sum_15_lag1', 'realized_vol_std_sum_240_lag1', 'm15_15mean_lag1', 'm15_240mean_lag1', 'm15_15std_lag1', 'vol240std_lag1', 'c240sum_lag1', 'log_ret_lag2', 'realized_vol_15_lag2', 'realized_vol_240_lag2', 'realized_vol_std_sum_15_lag2', 'realized_vol_std_sum_240_lag2', 'm15_15mean_lag2', 'm15_240mean_lag2', 'm15_15std_lag2', 'vol240std_lag2']
['log_r', 'log_ret', 'realized_vol_15', 'realized_vol_240', 'log_ret_15_lag', 'ENV_up', 'ENV_do', 'm15_15mean', 'm15_240mean', 'm15_15std', 'log_ret_lag1', 'realized_vol_15_lag1', 'realized_vol_240_lag1', 'realized_vol_std_sum_15_lag1', 'realized_vol_std_sum_240_

In [28]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print('Trainm前')
print(mem.percent)
print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
print('Peak:', peak_memory()/10**6, 'GB')

Trainm前
30.0
Use Memory total: 5.273816299438477 GiB
Peak: 10.66072 GB


In [29]:
def weighted_correlation(a, b, weights):

  '''
  a : pred
  b : target
  c : weight
  '''  
  w = np.ravel(weights)
  a = np.ravel(a)
  b = np.ravel(b)

  sum_w = np.sum(w)
  mean_a = np.sum(a * w) / sum_w
  mean_b = np.sum(b * w) / sum_w
  var_a = np.sum(w * np.square(a - mean_a)) / sum_w
  var_b = np.sum(w * np.square(b - mean_b)) / sum_w

  cov = np.sum((a * b * w)) / np.sum(w) - mean_a * mean_b
  corr = cov / np.sqrt(var_a * var_b)

  return corr

##metric
# https://stackoverflow.com/questions/38641691/weighted-correlation-coefficient-with-pandas
def wmean(x, w):
    return np.sum(x * w) / np.sum(w)

def wcov(x, y, w):
    return np.sum(w * (x - wmean(x, w)) * (y - wmean(y, w))) / np.sum(w)

def wcorr(x, y, w):
    return wcov(x, y, w) / np.sqrt(wcov(x, x, w) * wcov(y, y, w))

def eval_wcorr(preds, train_data):
    w = train_data.add_w.values.flatten()
    y_true = train_data.get_label()
    return 'eval_wcorr', wcorr(preds, y_true, w), True

# vizualize
def visualize_importance(models, feat_train_df):
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importance(importance_type="gain")
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(len(order) * .4, 7))
    sns.boxenplot(data=feature_importance_df, x='column', y='feature_importance', order=order, ax=ax, palette='viridis')
    ax.tick_params(axis='x', rotation=90)
    fig.tight_layout()
    return fig, ax, feature_importance_df

In [30]:
#features = [col for col in train.columns if col not in {'timestamp', 'Target', 'Target_M','weights', 'fold', 'rang'}]


#valid_oof = np.zeros((valid.shape[0],))
#valid_oof2 = np.zeros((valid.shape[0],))
#valid_oof3= np.zeros((valid.shape[0],))

models = []
models2 = []
models3 = []
models4 = []

n_fold = 1

if VALID:
    timest = valid['timestamp']
    weights_tr = pd.merge(train[['Asset_ID']], asset_details[['Asset_ID', 'Weight']], on='Asset_ID', how='left')['Weight']
    weights_val = pd.merge(valid[['Asset_ID']], asset_details[['Asset_ID', 'Weight']], on='Asset_ID', how='left')['Weight']
    
    for fold in range(n_fold):
        evals_result = {}
        #print(fold)
        ind_t = train.index
        y_train = train['Target'].values# * 1000 
        train = train[features]
        train.fillna(-999, inplace=True)
        train.replace(np.inf, -999, inplace=True)

        #y_train.fillna(-999, inplace=True)
        #y_train.replace(np.inf, -999, inplace=True)

        train['Asset_ID'] = train['Asset_ID'].astype('category')

        ind_v = valid.index
        y_val = valid['Target'].values #* 1000
        valid = valid[features]
        valid.fillna(-999, inplace=True)
        valid.replace(np.inf, -999, inplace=True)


        valid['Asset_ID'] = valid['Asset_ID'].astype('category')

        #weights_tr_fold = weights_tr[ind_t].reset_index(drop=True, inplace=True)
        #weights_val_fold = weights_val[ind_v].reset_index(drop=True, inplace=True)
        mem = psutil.virtual_memory() 
        print('Trainm前')
        print(mem.percent)
        print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
        print('Peak:', peak_memory()/10**6, 'GB')
        print("="*100)


        train_dataset = lgb.Dataset(train, y_train, feature_name = features, categorical_feature= ['Asset_ID'])
        val_dataset = lgb.Dataset(valid, y_val, feature_name = features, categorical_feature= ['Asset_ID'])
        #train_dataset.add_w = weights_tr_fold
        #val_dataset.add_w = weights_val_fold
        train_dataset.add_w = weights_tr
        val_dataset.add_w = weights_val
        #weights_train = train[['weights']]
        #weights_test = test[['weights']]

        #scaler = StandardScaler()
        #scaler.fit(x_train)

        #x_train = scaler.transform(x_train)
        #print(x_train.shape)
        print('A')
        # メモリ使用率を取得
        mem = psutil.virtual_memory() 
        print('Trainm前')
        print(mem.percent)
        print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
        print('Peak:', peak_memory()/10**6, 'GB')

        params = {
            'objective': 'regression',  #objectives = ['regression','regression_l1', 'huber', 'fair','quantile', 'mape', 'gamma','tweedie']
            #'fair_c': 100,
            #'metric': 'None',
            'boosting_type': 'gbdt',
            #'max_bin':255,
            'min_data_in_leaf':150,
            'subsample': 0.7,
            'subsample_freq': 5, # 何回に一回バギングするか
            #'feature_fraction': 0.4,
            'max_depth': -1,
            #'num_leaves':32, 
            'learning_rate': 0.05,
            #'subsample_freq': 4,
            #'feature_fraction': 0.4,
            'colsample_bytree': 0.9, 
            'lambda_l1': 10,
            'lambda_l2': 10,
            'seed': 1991,
            'verbose': -1,
            'n_jobs':-1
            }

        model = lgb.train(params = params,
                              num_boost_round = 300, 
                              train_set = train_dataset, 
                              valid_sets = [val_dataset],
                              #early_stopping_rounds=20,
                              verbose_eval = 50,
                              feval=eval_wcorr,
                              evals_result = evals_result 
                             )

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        val_pred = model.predict(valid)
        #valid_oof[ind_v] = val_pred
        score = weighted_correlation(val_pred, y_val, weights_val)
        models.append(model)
        print("Fold metric score:", score)
        print("Fold single corr:", np.corrcoef(val_pred, y_val)[0][1])
        print(pearsonr(y_val, val_pred))
        print("="*100)

        del train_dataset
        del val_dataset
        gc.collect()

        train_dataset = lgb.Dataset(train, y_train, feature_name = features, categorical_feature= ['Asset_ID'])
        val_dataset = lgb.Dataset(valid, y_val, feature_name = features, categorical_feature= ['Asset_ID'])
        train_dataset.add_w = weights_tr
        val_dataset.add_w = weights_val

        params2 = {
            'objective': 'regression',  #objectives = ['regression','regression_l1', 'huber', 'fair','quantile', 'mape', 'gamma','tweedie']
            #'fair_c': 100,
            #'metric': 'None',
            'boosting_type': 'gbdt',
            #'max_bin':255,
            'min_data_in_leaf':150,
            'subsample': 0.7,
            'subsample_freq': 5, # 何回に一回バギングするか
            #'feature_fraction': 0.4,
            'max_depth': -1,
            #'num_leaves':32, 
            'learning_rate': 0.05,
            #'subsample_freq': 4,
            #'feature_fraction': 0.4,
            'colsample_bytree': 0.9, 
            'lambda_l1': 10,
            'lambda_l2': 10,
            'seed': 91,
            'verbose': -1,
            'n_jobs':-1
            }

        model = lgb.train(params = params2,
                              num_boost_round = 300, 
                              train_set = train_dataset, 
                              valid_sets = [val_dataset],
                              #early_stopping_rounds=20,
                              verbose_eval = 50,
                              feval=eval_wcorr,
                              evals_result = evals_result 
                             )

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        val_pred2 = model.predict(valid)
        #valid_oof2[ind_v] = val_pred2
        score = weighted_correlation(val_pred2, y_val, weights_val)
        models2.append(model)
        print("Fold metric score:", score)
        print("Fold single corr:", np.corrcoef(val_pred2, y_val)[0][1])
        print(pearsonr(y_val, val_pred2))
        print("="*100)


        del train_dataset
        del val_dataset
        gc.collect()

        train_dataset = lgb.Dataset(train, y_train, feature_name = features, categorical_feature= ['Asset_ID'])
        val_dataset = lgb.Dataset(valid, y_val, feature_name = features, categorical_feature= ['Asset_ID'])
        train_dataset.add_w = weights_tr
        val_dataset.add_w = weights_val

        params3 = {
            'objective': 'regression',  #objectives = ['regression','regression_l1', 'huber', 'fair','quantile', 'mape', 'gamma','tweedie']
            #'fair_c': 100,
            #'metric': 'None',
            'boosting_type': 'gbdt',
            #'max_bin':255,
            'min_data_in_leaf':150,
            'subsample': 0.7,
            'subsample_freq': 5, # 何回に一回バギングするか
            #'feature_fraction': 0.4,
            'max_depth': -1,
            #'num_leaves':32, 
            'learning_rate': 0.051,
            #'subsample_freq': 4,
            #'feature_fraction': 0.4,
            'colsample_bytree': 0.9, 
            'lambda_l1': 10,
            'lambda_l2': 10,
            'seed': 31,
            'verbose': -1,
            'n_jobs':-1
            }

        model = lgb.train(params = params3,
                              num_boost_round = 300, 
                              train_set = train_dataset, 
                              valid_sets = [val_dataset],
                              #early_stopping_rounds=20,
                              verbose_eval = 50,
                              feval=eval_wcorr,
                              evals_result = evals_result 
                             )

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        val_pred3 = model.predict(valid)
        #valid_oof3[ind_v] = val_pred3
        score = weighted_correlation(val_pred3, y_val, weights_val)
        models3.append(model)
        print("Fold metric score:", score)
        print("Fold single corr:", np.corrcoef(val_pred3, y_val)[0][1])
        print(pearsonr(y_val, val_pred3))
        print("="*100)

        del train_dataset
        del val_dataset
        gc.collect()

        ################################# ridge #################################
        
        model4 = Ridge(alpha=10)
        # model = lgb.LGBMRegressor(n_jobs=-1, random_state=1)

        model4 = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model4)
        ])

        model4.fit(train[features_ridge].values, y_train)

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        val_pred4 = model4.predict(valid[features_ridge].values)
        #valid_oof[ind_v] = val_pred
        score = weighted_correlation(val_pred4, y_val, weights_val)
        models4.append(model4)
        print("Fold metric score:", score)
        print("Fold single corr:", np.corrcoef(val_pred4, y_val)[0][1])
        print(pearsonr(y_val, val_pred4))
        print("="*100)

else:
    weights_tr = pd.merge(train[['Asset_ID']], asset_details[['Asset_ID', 'Weight']], on='Asset_ID', how='left')['Weight']
    #weights_val = pd.merge(valid[['Asset_ID']], asset_details[['Asset_ID', 'Weight']], on='Asset_ID', how='left')['Weight']
    for fold in range(n_fold):
        evals_result = {}
        #print(fold)
        ind_t = train.index
        y_train = train['Target'].values# * 1000 
        train = train[features]
        train.fillna(-999, inplace=True)
        train.replace(np.inf, -999, inplace=True)

        #y_train.fillna(-999, inplace=True)
        #y_train.replace(np.inf, -999, inplace=True)

        train['Asset_ID'] = train['Asset_ID'].astype('category')

        #ind_v = valid.index
        #y_val = valid['Target'].values #* 1000
        #valid = valid[features]
        #valid.fillna(-999, inplace=True)
        #valid.replace(np.inf, -999, inplace=True)
        #valid['Asset_ID'] = valid['Asset_ID'].astype('category')

        #weights_tr_fold = weights_tr[ind_t].reset_index(drop=True, inplace=True)
        #weights_val_fold = weights_val[ind_v].reset_index(drop=True, inplace=True)
        mem = psutil.virtual_memory() 
        print('Trainm前')
        print(mem.percent)
        print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
        print('Peak:', peak_memory()/10**6, 'GB')
        print("="*100)


        train_dataset = lgb.Dataset(train, y_train, feature_name = features, categorical_feature= ['Asset_ID'])
        #val_dataset = lgb.Dataset(valid, y_val, feature_name = features, categorical_feature= ['Asset_ID'])
        #train_dataset.add_w = weights_tr_fold
        #val_dataset.add_w = weights_val_fold
        train_dataset.add_w = weights_tr
        #val_dataset.add_w = weights_val
        #weights_train = train[['weights']]
        #weights_test = test[['weights']]

        #scaler = StandardScaler()
        #scaler.fit(x_train)

        #x_train = scaler.transform(x_train)
        #print(x_train.shape)
        print('A')
        # メモリ使用率を取得
        mem = psutil.virtual_memory() 
        print('Trainm前')
        print(mem.percent)
        print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
        print('Peak:', peak_memory()/10**6, 'GB')

        params = {
            'objective': 'regression',  #objectives = ['regression','regression_l1', 'huber', 'fair','quantile', 'mape', 'gamma','tweedie']
            #'fair_c': 100,
            #'metric': 'None',
            'boosting_type': 'gbdt',
            #'max_bin':255,
            'min_data_in_leaf':150,
            'subsample': 0.7,
            'subsample_freq': 5, # 何回に一回バギングするか
            #'feature_fraction': 0.4,
            'max_depth': -1,
            #'num_leaves':32, 
            'learning_rate': 0.01,
            #'subsample_freq': 4,
            #'feature_fraction': 0.4,
            'colsample_bytree': 0.9, 
            'lambda_l1': 10,
            'lambda_l2': 10,
            'seed': 1991,
            'verbose': -1,
            'n_jobs':-1
            }

        model = lgb.train(params = params,
                              num_boost_round = 350, 
                              train_set = train_dataset, 
                              #valid_sets = [val_dataset],
                              #early_stopping_rounds=20,
                              verbose_eval = 50,
                              feval=eval_wcorr,
                              evals_result = evals_result 
                             )

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        #val_pred = model.predict(valid)
        #valid_oof[ind_v] = val_pred
        #score = weighted_correlation(val_pred, y_val, weights_val)
        models.append(model)
        #print("Fold metric score:", score)
        #print("Fold single corr:", np.corrcoef(val_pred, y_val)[0][1])
        #print(pearsonr(y_val, val_pred))
        print("="*100)

        del train_dataset
        #del val_dataset
        gc.collect()

        train_dataset = lgb.Dataset(train, y_train, feature_name = features, categorical_feature= ['Asset_ID'])
        #val_dataset = lgb.Dataset(valid, y_val, feature_name = features, categorical_feature= ['Asset_ID'])
        train_dataset.add_w = weights_tr
        #val_dataset.add_w = weights_val

        params2 = {
            'objective': 'regression',  #objectives = ['regression','regression_l1', 'huber', 'fair','quantile', 'mape', 'gamma','tweedie']
            #'fair_c': 100,
            #'metric': 'None',
            'boosting_type': 'gbdt',
            #'max_bin':255,
            'min_data_in_leaf':150,
            'subsample': 0.7,
            'subsample_freq': 5, # 何回に一回バギングするか
            #'feature_fraction': 0.4,
            'max_depth': -1,
            #'num_leaves':32, 
            'learning_rate': 0.01,
            #'subsample_freq': 4,
            #'feature_fraction': 0.4,
            'colsample_bytree': 0.9, 
            'lambda_l1': 10,
            'lambda_l2': 10,
            'seed': 91,
            'verbose': -1,
            'n_jobs':-1
            }

        model = lgb.train(params = params2,
                              num_boost_round = 350, 
                              train_set = train_dataset, 
                              #valid_sets = [val_dataset],
                              #early_stopping_rounds=20,
                              verbose_eval = 50,
                              feval=eval_wcorr,
                              evals_result = evals_result 
                             )

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        #val_pred2 = model.predict(valid)
        #valid_oof2[ind_v] = val_pred2
        #score = weighted_correlation(val_pred2, y_val, weights_val)
        models2.append(model)
        #print("Fold metric score:", score)
        #print("Fold single corr:", np.corrcoef(val_pred2, y_val)[0][1])
        #print(pearsonr(y_val, val_pred2))
        print("="*100)


        del train_dataset
        #del val_dataset
        gc.collect()

        train_dataset = lgb.Dataset(train, y_train, feature_name = features, categorical_feature= ['Asset_ID'])
        #val_dataset = lgb.Dataset(valid, y_val, feature_name = features, categorical_feature= ['Asset_ID'])
        train_dataset.add_w = weights_tr
        #val_dataset.add_w = weights_val

        params3 = {
            'objective': 'regression',  #objectives = ['regression','regression_l1', 'huber', 'fair','quantile', 'mape', 'gamma','tweedie']
            #'fair_c': 100,
            #'metric': 'None',
            'boosting_type': 'gbdt',
            #'max_bin':255,
            'min_data_in_leaf':150,
            'subsample': 0.7,
            'subsample_freq': 5, # 何回に一回バギングするか
            #'feature_fraction': 0.4,
            'max_depth': -1,
            #'num_leaves':32, 
            'learning_rate': 0.01,
            #'subsample_freq': 4,
            #'feature_fraction': 0.4,
            'colsample_bytree': 0.9, 
            'lambda_l1': 10,
            'lambda_l2': 10,
            'seed': 31,
            'verbose': -1,
            'n_jobs':-1
            }

        model = lgb.train(params = params3,
                              num_boost_round = 350, #int(300*1.1), 
                              train_set = train_dataset, 
                              #valid_sets = [val_dataset],
                              #early_stopping_rounds=20,
                              verbose_eval = 50,
                              feval=eval_wcorr,
                              evals_result = evals_result 
                             )

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        #val_pred3 = model.predict(valid)
        #valid_oof3[ind_v] = val_pred3
        #score = weighted_correlation(val_pred3, y_val, weights_val)
        models3.append(model)
        #print("Fold metric score:", score)
        #print("Fold single corr:", np.corrcoef(val_pred3, y_val)[0][1])
        #print(pearsonr(y_val, val_pred3))
        print("="*100)

        del train_dataset
        #del val_dataset
        gc.collect()

        ################################# ridge #################################
        '''
        model4 = Ridge(alpha=10)
        # model = lgb.LGBMRegressor(n_jobs=-1, random_state=1)

        model4 = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model4)
        ])

        model4.fit(train[features_ridge].values, y_train)

        #model = Ridge(alpha=5)
        #model.fit(x_train, y_train)

        #x_val = scaler.transform(x_val)
        #val_pred4 = model4.predict(valid[features_ridge].values)
        #valid_oof[ind_v] = val_pred
        #score = weighted_correlation(val_pred4, y_val, weights_val)
        models4.append(model4)
        #print("Fold metric score:", score)
        #print("Fold single corr:", np.corrcoef(val_pred4, y_val)[0][1])
        #print(pearsonr(y_val, val_pred4))
        #print("="*100) 
        '''
        for i in range(14):
            model4 = Ridge(alpha=10)
            # model = lgb.LGBMRegressor(n_jobs=-1, random_state=1
            print('ASSET : ', i)
            model4 = Pipeline([
                ('scaler', StandardScaler()), 
                ('model', model4)
            ])
            indt = train[train['Asset_ID' ] == i].index
            #indv = valid[valid['Asset_ID' ] == i].index
            model4.fit(train.iloc[indt][features_ridge].values, y_train[indt])
            #val_pred4 = model4.predict(valid.iloc[indv][features_ridge].values)
            #valid_oof[ind_v] = val_pred
            #score = weighted_correlation(val_pred4, y_val[indv], weights_val[indv])
            models4.append(model4)
            #valid_oofridge[indv] = val_pred4
            #print("Fold metric score:", score)
            #print("Fold single corr:", np.corrcoef(val_pred4, y_val[indv])[0][1])
            #print(pearsonr(y_val[indv], val_pred4))
            #print('='  * 100)


Trainm前
45.8
Use Memory total: 8.051359550476073 GiB
Peak: 10.66072 GB
A
Trainm前
45.8
Use Memory total: 8.051359550476073 GiB
Peak: 10.66072 GB


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


ASSET :  0
ASSET :  1
ASSET :  2
ASSET :  3
ASSET :  4
ASSET :  5
ASSET :  6
ASSET :  7
ASSET :  8
ASSET :  9
ASSET :  10
ASSET :  11
ASSET :  12
ASSET :  13


In [31]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print('Train後')
print(mem.percent)
print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
print('Peak:', peak_memory()/10**6, 'GB')

Train後
52.7
Use Memory total: 9.264337299346924 GiB
Peak: 14.792868 GB


In [32]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print(mem.percent)
print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
print('Peak:', peak_memory()/10**6, 'GB')

52.7
Use Memory total: 9.264337299346924 GiB
Peak: 14.792868 GB


In [33]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print(mem.percent)
print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
print('Peak:', peak_memory()/10**6, 'GB')

52.7
Use Memory total: 9.264337299346924 GiB
Peak: 14.792868 GB


In [34]:
del train
gc.collect()

if VALID:
    del valid
    gc.collect

In [35]:
print(features)
print(len(features))

['Asset_ID', 'HperL', 'log_r', 'log_ret', 'realized_vol_15', 'realized_vol_240', 'log_ret_15_lag', 'log_ret_15_std', 'ENV_up', 'ENV_do', 'RSI15', 'RSI240', 'macd60', 'signal15', 'signal60', 'realized_vol_std_sum_15', 'realized_vol_std_sum_240', 'm15_15mean', 'm15_240mean', 'm15_15std', 'vol240std', 'c240sum', 'log_ret_lag1', 'realized_vol_15_lag1', 'realized_vol_240_lag1', 'realized_vol_std_sum_15_lag1', 'realized_vol_std_sum_240_lag1', 'm15_15mean_lag1', 'm15_240mean_lag1', 'm15_15std_lag1', 'vol240std_lag1', 'c240sum_lag1', 'log_ret_lag2', 'realized_vol_15_lag2', 'realized_vol_240_lag2', 'realized_vol_std_sum_15_lag2', 'realized_vol_std_sum_240_lag2', 'm15_15mean_lag2', 'm15_240mean_lag2', 'm15_15std_lag2', 'vol240std_lag2']
41


# Inference

In [36]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print('Test前')
print(mem.percent)
print(f'Use Memory total: {mem_bytes * (mem.percent / 100) / (2**30)} GiB')
print('Peak:', peak_memory()/10**6, 'GB')

Test前
36.9
Use Memory total: 6.486794048309326 GiB
Peak: 14.792868 GB


In [37]:
ensweight = np.array([1,1,2,1,1,2,2,2,1,2,1,2,1,1])
print(len(ensweight))

14


In [38]:
%%time

#not building the weights each loops
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
dict_weights = {}

for i in range(asset_details.shape[0]):
    dict_weights[asset_details.iloc[i,0]] = asset_details.iloc[i,1]
weigths = np.array([dict_weights[i] for i in range(14)])

# only needed when saving ?
dtype={'Asset_ID': 'int8', 'Count': 'int32', 'row_id': 'int32', 'Count': 'int32',
       'Open': 'float32', 'High': 'float32', 'Low': 'float32', 'Close': 'float32',
       'Volume': 'float32', 'VWAP': 'float32'}
#test_df = test_df.astype(dtype)

#refactoring functions:

# #instantiation Moving average features dict
# dict_RM = {}
# dict_RM_M = {}

# for lag in MA_lags:
#     dict_RM[lag] = RunningMean(lag)
#     dict_RM_M[lag] = RunningMean(lag)
    


# #instantiation dict betas
# dict_MM = {}
# dict_Mr = {}

#dict_RM = pickle.load(open('../input/on-line-feature-engineering/dict_RM_4.pkl', 'rb'))
#dict_RM_M = pickle.load(open('../input/on-line-feature-engineering/dict_RM_M_4.pkl', 'rb'))
#dict_MM = pickle.load(open('../input/on-line-feature-engineering/dict_MM_4.pkl', 'rb'))
#dict_Mr = pickle.load(open('../input/on-line-feature-engineering/dict_MR_4.pkl', 'rb'))


# for lag in beta_lags:
#     dict_MM[lag] = RunningMean(lag)
#     dict_Mr[lag] = RunningMean(lag)


env = gresearch_crypto.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    #print('A')
    try:
        test_df = test_df.fillna(0)

        timestamp,Asset_ID,Count,O,H,L,C,Volume,VWAP,row_id = Clean_df(test_df.values)
        #print('B')
        # np.array([Count,O,H,L,C,Price,Volume,VWAP,Dollars,Volume_per_trade,Dollars_per_trade,log_ret,GK_vol,RS_vol])
        Features = Base_Feature_fn(timestamp,Asset_ID,Count,O,H,L,C,Volume,VWAP)
        baseinfo_features = np.delete(Features, np.s_[0:6], axis=1)
        Features = np.delete(Features, np.s_[-8:], axis=1) 
        #print('C')   

        #removing wieghts when data is missing so that they don't appears in market
        weigthss = np.where(np.isnan(O),O,weigths) # ここ変えた
        #Market_Features = np.nansum(Features*np.expand_dims(weigths,axis=1)/np.nansum(weigths),axis=0)
        #Market_Features = np.tile(Market_Features,(14,1))

        #np.array((sin_month,cos_month,sin_day,cos_day,sin_hour,cos_hour,sin_minute,cos_minute))
        time = timestamp_to_date(timestamp[0])
        Time_Features = Time_Feature_fn(time)
        #Time_Features = np.tile(Time_Features,(14,1))

        MA_Features = []
        #MA_Features_M  = []
        #MA_Features_max = [] # max featur
        #MA_Features_min = [] # min feature
        MA_Features_maxmin = []
        RealV_Feature = []
        RealV_Feature_15240 = []
        last_log_r_Feature = []
        Std_Features = []
        Richman_Features = []
        Richman_Features_beata = []
        Btc_eth_corr_Features = []
        Updo_rate_Features = []
        MACD_Featuees = []
        Market_Features = []
        Volume_Features = []
        Other_teq_Features = []
        teq_lagfeatures = []
        RSI_features = []

        for lag in MA_lags:
            dict_RM[lag].push(Features)
            #dict_RM_M[lag].push(Market_Features)
            dict_RM_base[lag].push(np.concatenate((baseinfo_features, weigthss.reshape(14,1)), axis=1)) # ここ変えた
            #dict_RM_richbeta
            #MA_Features.append(dict_RM[lag].get_mean())
            #MA_Features_M.append(dict_RM_M[lag].get_mean())
            #RealV_Feature.append(dict_RM[lag].get_realvol()) 


        last_log_r_Feature.append(dict_RM_base[15].get_lag()) # ここ
        Std_Features.append(dict_RM[15].get_std())

        beta_calc_features, rich_features = dict_RM_base[15].rich_cldiff() # richman feature1　ここ
        Richman_Features.append(rich_features)


        dict_RM_17[17].push(baseinfo_features[:,0].reshape(14,1))
        rich_tech_fe = dict_RM_17[17].rich_beta()
        Richman_Features_beata.append(rich_tech_fe)
        #print('A')

        dict_RM_richbeta[3601].push(baseinfo_features[:,0].reshape(14,1)) # richman feature2
        macd_fe, rsi_fe = dict_RM_richbeta[3601].make_rsi()
        RSI_features.append(rsi_fe)
        #print('A')
        #rich_tech_fe, macd_fe = dict_RM_richbeta[3601].rich_beta()



        #print(Richman_Features_beata)

        dict_RM_macd[10].push(macd_fe)
        #print('A')
        MACD_Featuees.append(np.concatenate((np.delete(macd_fe, np.s_[0], axis=1), dict_RM_macd[10].macd()), axis=1))
        #print('A')

        #Btc_eth_corr_Features.append(dict_RM_base[15].corr_btc_eth())
        #Updo_rate_Features.append(dict_RM_base[15].updown_rate())

        RealV_Feature.append(dict_RM[240].get_realvol())  # vola
        #print('A')

        dict_RM_vols[240].push(dict_RM[15].get_realvol())
        RealV_Feature_15240.append(dict_RM_vols[240].vol_sum_std())

        market_mean = np.nanmean(Features[:,5])
        market_mean_abs = np.nanmean(abs(Features[:,5]))
        dict_RM_market[240].push(np.array([market_mean, market_mean_abs]))
        Market_Features.append(dict_RM_market[240].mmean_std())

        Volume_Features.append(dict_RM_base[240].vol_features())





        #Other_teq_Features.append(dict_RM_base[240].oth_teq())
        #print('zzz')
        #RealV_Feature.append(dict_RM[60].get_realvol())

        #for term in [60, 240, 1440]:
        #  MA_Features_maxmin.append(dict_RM_base[term].get_maxmin())# maxmin
          #print('A')


        #print('A')
        #MA_Features = np.concatenate(MA_Features,axis=1)
        #print('B')
        #MA_Features_M = np.concatenate(MA_Features_M)
        #MA_Features_maxmin = np.concatenate(MA_Features_maxmin,axis=1) # maxmin
        #print('C')

        #MA_Features_max = np.concatenate(MA_Features_max,axis=1)
        #MA_Features_min = np.concatenate(MA_Features_min,axis=1)
        RealV_Feature = np.concatenate(RealV_Feature,axis=1) # vola

        last_log_r_Feature = np.concatenate(last_log_r_Feature,axis=1) # lag
        Std_Features = np.concatenate(Std_Features,axis=1) # 標準偏差

        #Richman_Features = np.concatenate(Richman_Features, axis=1) # richman feature1
        Richman_Features_beata = np.concatenate(Richman_Features_beata, axis=1) # richman feature1
        RSI_features = np.concatenate(RSI_features, axis=1)
        MACD_Featuees = np.concatenate(MACD_Featuees, axis=1)

        RealV_Feature_15240 = np.concatenate(RealV_Feature_15240, axis=1)

        Market_Features = np.concatenate(Market_Features, axis=1)
        Volume_Features = np.concatenate(Volume_Features, axis=1)




        dict_RM_lag[5].push(np.concatenate((np.delete(Features, np.s_[:-1], axis=1),  RealV_Feature, RealV_Feature_15240, Market_Features, Volume_Features), axis=1))
        #print('A')
        teq_lagfeatures.append(dict_RM_lag[5].tec_lags())
        teq_lagfeatures = np.concatenate(teq_lagfeatures, axis=1)



        Features_ridge = Features[:, [3, 5]]
        teq_lagfeatures_ridge = np.delete(teq_lagfeatures, np.s_[9], axis=1)

        Features = np.delete(Features, np.s_[1:3], axis=1)
        Features = np.delete(Features, 2, axis=1)
        #print('C')
        #betas = np.concatenate(betas,axis=1)
        #betas = np.nan_to_num(betas, nan=0., posinf=0., neginf=0.) 
        #Features = np.delete(Features, 4, axis=1)
        #values = np.concatenate((Features,np.tile(Market_Features,(14,1)),np.tile(Time_Features,(14,1)),MA_Features,np.tile(MA_Features_M,(14,1)),betas, MA_Features_max, MA_Features_min, RealV_Feature, np.expand_dims(Target,axis=1)),axis=1)
        #values = np.concatenate((Features,RealV_Feature,betas,MA_Features, MA_Features_maxmin,np.expand_dims(Target,axis=1)),axis=1)

        values = np.concatenate((np.expand_dims(Asset_ID,axis=1), Features,RealV_Feature,last_log_r_Feature, Std_Features, #Richman_Features,
                                              Richman_Features_beata, RSI_features,
                                              MACD_Featuees, RealV_Feature_15240, Market_Features, 
                                              Volume_Features, #Other_teq_Features, 
                                              teq_lagfeatures, 
                                              ),axis=1)#values = np.concatenate((Features,RealV_Feature,last_log_r_Feature, Std_Features, Richman_Features,Richman_Features_beata, MACD_Featuees, Btc_eth_corr_Features, Updo_rate_Features, np.expand_dims(Target,axis=1)),axis=1)

        #values = np.delete(values, np.s_[44:], axis=1)
        values[np.isnan(values)] = -999
        values[np.isnan(values)] = -999
        values[values == np.inf] = -999


        values_ridge = np.concatenate((Features_ridge,RealV_Feature,last_log_r_Feature, #Std_Features, #Richman_Features,
                                       Richman_Features_beata, #RSI_features,
                                       #MACD_Featuees, 
                                       #RealV_Feature_15240, 
                                       Market_Features, 
                                       #Volume_Features, #Other_teq_Features, 
                                       teq_lagfeatures_ridge, 
                                       ),axis=1)#values = np.concatenate((Features,RealV_Feature,last_log_r_Feature, Std_Features, Richman_Features,Richman_Features_beata, MACD_Featuees, Btc_eth_corr_Features, Updo_rate_Features, np.expand_dims(Target,axis=1)),axis=1)

        #values = np.delete(values, np.s_[44:], axis=1)
        values_ridge[np.isnan(values_ridge)] = -999
        values_ridge[np.isnan(values_ridge)] = -999
        values_ridge[values_ridge == np.inf] = -999



        # diff features
        #basep = [1,2,3,4,5]
        #llag = [[19,20,21,22,23], [24,25,26,27,28]]
        #for col in tqdm(range(len(basep))):
        #    for lagi in range(len(llag)):
            #print(col, lagf)
        #        values[:,basep[col]] - values[:,llag[lagi][col]]

        preds1 = (models[0].predict(values))
        preds2 = (models2[0].predict(values))
        preds3 = (models3[0].predict(values))

        preds = (preds1 + preds2 + preds3) / 3
        
        preds_ridge = []
        preds_final = []
        for i in range(14):
            pre = models4[i].predict(values_ridge)
            
            if ensweight[i] == 1:
                preds_final.append(preds[i])
            
            else:
                preds_final.append(pre[i])
            #preds_ridge.append(pre[i])
        #preds_ridge = (models4[0].predict(values_ridge))
        
        
        #for i in range(14):
        #    if ensweight[i] == 1:
        #        preds_final.append(preds[i])
        #    else:
        #        preds_final.append(preds_ridge[i])
                
        preds_final = np.array(preds_final)
        #weightv = 0.4
        #preds = (preds * weightv) + (1-weightv) * np.array(preds_ridge)
        #preds = preds * weight + (1-weight) * preds_ridge
        #preds = np.array([model.predict(values) for model in models])
        #preds = np.median(np.expm1(preds), axis=0)
        #print('I')
        #preds = model_lgbm[0].predict(values)
        #preds = model.predict(scaler.transform(values))
        #preds = np.median(np.array([model_lgbm[str(i)+'-'+str(j)].predict(values, num_iteration = ES_it[str(i)+'-'+str(j)]) for i in range(5) for j in range(5)]),axis=0)

        sample_prediction_df['Target'] = [preds_final[(row_id == rid)][0] for rid in sample_prediction_df.row_id.values]
        env.predict(sample_prediction_df)
    
    except:
        a = np.array([ 2.08572321e-05,  4.94754720e-06, -3.10617725e-05,  1.57531076e-07,
                      -3.63379728e-06, -1.23053704e-05,  1.12755761e-05, -2.44581399e-05,
                      8.72486649e-05,  1.34530341e-05, -5.19872381e-08, -1.15822545e-05,
                      5.64495861e-07, -1.69252929e-05])

        sample_prediction_df['Target'] = a
        env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
CPU times: user 214 ms, sys: 1.96 ms, total: 216 ms
Wall time: 93.1 ms


In [39]:
sample_prediction_df

,row_id,Target
0,42,-0.000347
1,43,-0.016758
2,44,0.002395
3,45,0.000074
4,46,-0.000127
5,47,-0.008034
6,48,-0.030751
7,49,0.000619
8,50,-0.000324
9,51,-0.008875


In [40]:
# peak memory
import os
import re

def peak_memory():
    pid = os.getpid()
    with open(f'/proc/{pid}/status') as f:
        for line in f:
            if not line.startswith('VmHWM:'):
                continue
            return int(re.search('[0-9]+', line)[0])
    raise ValueError('Not Found')

def f():
    a = [0] * 20000000

print('before:', peak_memory()/10**6, 'GB')

before: 14.792868 GB
